# DATA PREPARATION AND PRE-PROCESSING

Using machine learning to predict NBA player improvement and future performances

I have always been a big fan of NBA and I always liked to look at NBA player stats and development of my favourite NBA players over the years. 
It is very important for NBA team management to access how well their players are developing for them to build a team to ultimately win a championship.
For my project, I hope to ultimately ultilise machine learning with python tools, to observe the logic behind how numbers can reflect the development and improvement of NBA players, and hope to predict the trend in the future. 
The dataset I acquired are 3 different parquet files, they are similar to csv files, they work the same in python, the compression and space occupancy for parquet is however much lower. The dataset I found is from Kaggle where the author did webscrapping from the basketball-reference.com. This website is reowned for storing all the NBA stats.
The dataset contains data from 1995 to 2023/2024 (Current NBA season).

In [47]:
#Importing all the tools and modules I might need
import os


import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns


import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from scipy.stats import norm

In [48]:
#Reading all the parquet files
average_df = pd.read_parquet('average.parq', engine='fastparquet')
advanced_df = pd.read_parquet('advanced.parq', engine='fastparquet')
roster_df = pd.read_parquet('roster.parq', engine='fastparquet')
pd.set_option('display.max_columns', None)

In [49]:
#Data Understanding

average_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,team_retcon
0,1,Isaiah Rider,23,75,67,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,MIN
1,2,Christian Laettner,25,81,80,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,MIN
2,3,Doug West,27,71,65,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,MIN
3,4,Tom Gugliotta,25,31,17,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,MIN
4,5,Sean Rooks,25,80,70,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,MIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12621,20,Ryan Arcidiacono,28,9,4,16.2,0.9,3.6,0.250,0.8,2.2,0.350,0.1,1.3,0.083,0.359,0.0,0.0,NaN,0.0,1.2,1.2,2.3,0.3,0.0,0.7,1.6,2.6,POR,2023/2024,POR
12622,21,John Butler,20,19,1,11.6,0.9,2.8,0.321,0.4,1.8,0.229,0.5,0.9,0.500,0.396,0.2,0.2,0.750,0.2,0.7,0.9,0.6,0.4,0.5,0.1,1.2,2.4,POR,2023/2024,POR
12623,22,Jabari Walker,20,56,0,11.1,1.5,3.5,0.419,0.3,1.0,0.286,1.2,2.5,0.472,0.460,0.6,0.8,0.756,0.9,1.5,2.3,0.6,0.2,0.2,0.5,1.2,3.9,POR,2023/2024,POR
12624,23,Keon Johnson,20,40,0,10.4,1.7,4.5,0.376,0.7,2.0,0.346,1.0,2.4,0.402,0.455,0.7,1.0,0.659,0.3,0.8,1.1,1.5,0.5,0.2,1.1,1.1,4.7,POR,2023/2024,POR


In [50]:
#Lets have a look at average_df first, as this will be our main dataframe with most of the important stats in it. 
average_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12626 entries, 0 to 12625
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           12626 non-null  int64  
 1   Player       12626 non-null  object 
 2   Age          12626 non-null  int64  
 3   G            12626 non-null  int64  
 4   GS           12626 non-null  int64  
 5   MP           12626 non-null  float64
 6   FG           12626 non-null  float64
 7   FGA          12626 non-null  float64
 8   FG%          12546 non-null  float64
 9   3P           12626 non-null  float64
 10  3PA          12626 non-null  float64
 11  3P%          10701 non-null  float64
 12  2P           12626 non-null  float64
 13  2PA          12626 non-null  float64
 14  2P%          12495 non-null  float64
 15  eFG%         12546 non-null  float64
 16  FT           12626 non-null  float64
 17  FTA          12626 non-null  float64
 18  FT%          11969 non-null  float64
 19  ORB 

After looking at the average_df, I decided to make a data dictionary for average_df for clarity.

| Column name | Datatype | Description                                                                                   |
|-------------|----------|-----------------------------------------------------------------------------------------------|
| Player      | object   | First and last name of the NBA player                                                         |
| Age         | int64    | Age of the player during that season                                                          |
| G           | int64    | The number of games the player had played that season                                         |
| GS          | int64    | The number of games the player played as a starter                                            |
| MP          | float64  | The average minutes the player had played per game                                            |
| FG          | float64  | Field goals - The average number of times the player scored a basket per game                 |
| FGA         | float64  | Field goals attempted - The average number of times the player attempted to score a basket    |
| FG%         | float64  | The percentage of field goals made per game                                                   |
| 3P          | float64  | The number of 3-pointers made per game                                                        |
| 3PA         | float64  | The number of 3-pointers attempts made per game                                               |
| 3P%         | float64  | The percentage of 3-pointers made per game                                                    |
| 2P          | float64  | The number of 2-pointers made per game                                                        |
| 2PA         | float64  | The number of 2-pointers attempts made per game                                               |
| 2P%         | float64  | The percentage of 2-pointers made per game                                                    |
| eFG%        | float64  | Effective Field Goal Percentage; (FG + 0.5 * 3P)/FGA. Adjusts for 3-pointer being one more point than a 2-pointer |
| FT          | float64  | The number of free-throws made per game                                                       |
| FTA         | float64  | The number of free-throws attempts per game                                                   |
| FT%         | float64  | The percentage of free-throws made per game                                                   |
| ORB         | float64  | Offensive rebounds per game - When a player grabs a rebound from the opponents' basket        |
| DRB         | float64  | Defensive rebounds per game - When a player grabs a rebound from his own basket               |
| TRB         | float64  | Total rebounds - The sum of offensive and defensive rebounds the player grabbed per game      |
| AST         | float64  | Assists per game                                                                              |
| STL         | float64  | Steals per game                                                                               |
| BLK         | float64  | Blocks per game                                                                               |
| TOV         | float64  | Turnover per game - A turnover is when a player loses the ball to the opponent                |
| PF          | float64  | Personal fouls per game                                                                       |
| PTS         | float64  | Points per game                                                                               |
| team        | object   | The team name in abbreviation                                                                 |
| season      | object   | The year of that season started / the year of that season ended                               |
| team_retcon | object   | Team abbreviation                                                                             |


In [51]:
# Now lets look at the roster_df
roster_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12637 entries, 0 to 12636
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player       12637 non-null  object
 1   Pos          12637 non-null  object
 2   Ht           12637 non-null  object
 3   Wt           12637 non-null  int64 
 4   Birth Date   12637 non-null  object
 5   Unnamed: 6   12637 non-null  object
 6   Exp          12637 non-null  int64 
 7   College      11009 non-null  object
 8   team         12637 non-null  object
 9   season       12637 non-null  object
 10  team_retcon  12637 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.1+ MB


I made a data dictionary for roster_df too

| Column name | Datatype | Description                                                      |
|-------------|----------|------------------------------------------------------------------|
| Player      | object   | First and last name of the NBA player                            |
| Pos         | object   | Position of the player                                           |
| Ht          | object   | Height of the player in feet and inches                          |
| Wt          | int64    | Weight of the player in pounds                                   |
| Birth Date  | object   | Date of birth of the NBA player (Month dd, yyyy)                 |
| Unnamed: 6  | object   | The country the player is from                                   |
| Exp         | int64    | The number of years the player has played in the NBA             |
| College     | object   | The college of the U.S. players before joining the NBA           |
| team        | object   | The players' team name in abbreviation                           |
| season      | object   | The year of that season started / the year of that season ended  |
| team_retcon | object   | The players' team name in abbreviation

In [52]:
#Lastly lets look at the advanced_df
advanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12626 entries, 0 to 12625
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rk           12626 non-null  int64  
 1   Player       12626 non-null  object 
 2   Age          12626 non-null  int64  
 3   G            12626 non-null  int64  
 4   MP           12626 non-null  int64  
 5   PER          12622 non-null  float64
 6   TS%          12552 non-null  float64
 7   3PAr         12546 non-null  float64
 8   FTr          12546 non-null  float64
 9   ORB%         12622 non-null  float64
 10  DRB%         12622 non-null  float64
 11  TRB%         12622 non-null  float64
 12  AST%         12622 non-null  float64
 13  STL%         12622 non-null  float64
 14  BLK%         12622 non-null  float64
 15  TOV%         12563 non-null  float64
 16  USG%         12622 non-null  float64
 17  Unnamed: 17  0 non-null      float64
 18  OWS          12626 non-null  float64
 19  DWS 

The dataframe advanced_df gives us metrics that are calculated in a more advanced way with which includes comparing to team averages and league averages. I excluded columns that have mentioned previously in other df's dictionaries.

| Column name | Datatype | Description                                                                                                                                                                                      |
|-------------|----------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| MP          | int64      | Total minutes played by a player in a season.                                                                                                                                                    |
| PER         | float    | Player Efficiency Rating - The PER sums up all a player's positive accomplishments, subtracts the negative accomplishments, and returns a per-minute rating of a player's performance.           |
| TS%         | float    | True Shooting Percentage - a measure of shooting efficiency that takes into account field goals, 3-point field goals, and free throws.                                                           |
| 3PAr        | float    | 3-Point Attempt Rate- Percentage of total FG Attempts that are from 3-Point Range.                                                                                                               |
| FTr         | float    | Free Throw Rate - Measures Free Throws per field goal attempt.                                                                                                                                   |
| ORB%        | float    | Offensive Rebound Percentage - an estimate of the percentage of available offensive rebounds a player grabbed while he was on the floor.                                                         |
| DRB%        | float    | Defensive Rebound Percentage - an estimate of the percentage of available defensive rebounds a player grabbed while he was on the floor.                                                         |
| TRB%        | float    | Total Rebound Percentage - an estimate of the percentage of available rebounds a player grabbed while he was on the floor.                                                                       |
| AST%        | float    | Assist Percentage - The percentage of teammate field goals a player assisted on while they were on the floor                                                                                     |
| STL%        | float    | The ratio of a player's steals to the total number of opponent possessions expressed as a percentage.                                                                                            |
| BLK%        | float    | The percentage of opponent shots that a player blocks.                                                                                                                                           |
| TOV%        | float    | A player's TOV to his own team's possessions expressed as a percentage.                                                                                                                          |
| USG%        | float    | Usage Rate - an estimate of the percentage of a team's offensive possessions used by an individual player during his time on the floor.                                                          |
| OWS         | float    | Offensive Win Shares                                                                                                                                                                             |
| DWS         | float    | Defensive Win Shares                                                                                                                                                                             |
| WS          | float    | Winshares - a player statistic which attempts to divvy up credit for team success to the individuals on the team.                                                                                |
| WS/48       | float    | Win Shares Per 48 Minutes - an estimate of the number of wins contributed by the player per 48 minutes (league average is approximately 0.100).                                                  |
| OBPM        | float    | offensive box plus/minus                                                                                                                                                                         |
| DBPM        | float    | Defensive box plus/minus                                                                                                                                                                         |
| BPM         | float    | Box Plus/Minus - a basketball box score-based metric that estimates a basketball player's contribution to the team when that player is on the court.                                             |
| VORP        | float    | Value Over Replacement Player - a statistic used in basketball to measure a player's overall contributions to their team, when compared to an average "replacement" player at the same position. |


After looking at the three dataframes we will be working with and made data dictionaries for all of them, my next step is to merge them together to make a new dataframe which
only contains stats that will be important for our goal, and excluding the ones that aren't useful or existing already.

Starting with average_df, I decided to drop the "team_retcon" column first, as the data in "team_retcon" represents the same as "team"

In [53]:
new_average_df = average_df.drop(columns=['team_retcon']) #Removing the "team_retcon" column
new_average_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season
0,1,Isaiah Rider,23,75,67,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996
1,2,Christian Laettner,25,81,80,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996
2,3,Doug West,27,71,65,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996
3,4,Tom Gugliotta,25,31,17,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996
4,5,Sean Rooks,25,80,70,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12621,20,Ryan Arcidiacono,28,9,4,16.2,0.9,3.6,0.250,0.8,2.2,0.350,0.1,1.3,0.083,0.359,0.0,0.0,NaN,0.0,1.2,1.2,2.3,0.3,0.0,0.7,1.6,2.6,POR,2023/2024
12622,21,John Butler,20,19,1,11.6,0.9,2.8,0.321,0.4,1.8,0.229,0.5,0.9,0.500,0.396,0.2,0.2,0.750,0.2,0.7,0.9,0.6,0.4,0.5,0.1,1.2,2.4,POR,2023/2024
12623,22,Jabari Walker,20,56,0,11.1,1.5,3.5,0.419,0.3,1.0,0.286,1.2,2.5,0.472,0.460,0.6,0.8,0.756,0.9,1.5,2.3,0.6,0.2,0.2,0.5,1.2,3.9,POR,2023/2024
12624,23,Keon Johnson,20,40,0,10.4,1.7,4.5,0.376,0.7,2.0,0.346,1.0,2.4,0.402,0.455,0.7,1.0,0.659,0.3,0.8,1.1,1.5,0.5,0.2,1.1,1.1,4.7,POR,2023/2024


Moving to the roster_df, I want to get "Pos", "Ht", "Wt" and "Exp" from roster_df and add them into my average_df.
"Pos" - I believe the positions of the players are super important because different positions in NBA can have different stats on average, which will also link to a player's growth
depending on their position played.
"Ht" - The height of a player will also be important, linked to their positions played, but height of a player can also affect the potential of a player or a player's playstyle
"Wt" - The weight of the player can help determine a players' playstyle too, from agility to shooting performances etc.
"Exp" - The number of years a player has played can be important to determine and predict a players' growth potential.

In [54]:
roster_df_selected = roster_df[["Player", "team", "season", "Pos", "Ht", "Wt", "Exp"]] #Making a new roster DF with these columns in

In [55]:
roster_df_selected = roster_df[["Player", "team", "season", "Pos", "Ht", "Wt", "Exp"]]
combined_df = pd.merge(new_average_df, roster_df_selected, on=['Player', 'team', 'season'], how='left') #Adding the new roster DF to my average DF and making it a new DF called combined_df, 
#which will be our final dataframe we will be using as a whole.
combined_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp
0,1,Isaiah Rider,23,75,67,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1
1,2,Christian Laettner,25,81,80,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2
2,3,Doug West,27,71,65,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5
3,4,Tom Gugliotta,25,31,17,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2
4,5,Sean Rooks,25,80,70,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12621,20,Ryan Arcidiacono,28,9,4,16.2,0.9,3.6,0.250,0.8,2.2,0.350,0.1,1.3,0.083,0.359,0.0,0.0,NaN,0.0,1.2,1.2,2.3,0.3,0.0,0.7,1.6,2.6,POR,2023/2024,PG,6-3,195,5
12622,21,John Butler,20,19,1,11.6,0.9,2.8,0.321,0.4,1.8,0.229,0.5,0.9,0.500,0.396,0.2,0.2,0.750,0.2,0.7,0.9,0.6,0.4,0.5,0.1,1.2,2.4,POR,2023/2024,C,7-1,175,0
12623,22,Jabari Walker,20,56,0,11.1,1.5,3.5,0.419,0.3,1.0,0.286,1.2,2.5,0.472,0.460,0.6,0.8,0.756,0.9,1.5,2.3,0.6,0.2,0.2,0.5,1.2,3.9,POR,2023/2024,SF,6-9,215,0
12624,23,Keon Johnson,20,40,0,10.4,1.7,4.5,0.376,0.7,2.0,0.346,1.0,2.4,0.402,0.455,0.7,1.0,0.659,0.3,0.8,1.1,1.5,0.5,0.2,1.1,1.1,4.7,POR,2023/2024,SG,6-5,186,1


In [56]:
advanced_df

,Rk,Player,Age,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 17,OWS,DWS,WS,WS/48,Unnamed: 22,OBPM,DBPM,BPM,VORP,team,season,team_retcon
0,1,Christian Laettner,25,81,2770,17.7,0.580,0.043,0.543,7.3,19.7,13.6,15.5,1.9,2.4,16.5,22.5,NaN,3.6,2.6,6.2,0.108,NaN,0.7,0.4,1.1,2.2,MIN,1995/1996,MIN
1,2,Isaiah Rider,23,75,2645,15.6,0.548,0.317,0.271,4.2,7.3,5.8,18.7,1.4,0.7,14.2,28.2,NaN,1.4,0.4,1.9,0.034,NaN,2.0,-2.5,-0.5,1.0,MIN,1995/1996,MIN
2,3,Sean Rooks,25,80,2405,12.6,0.555,0.008,0.620,8.5,16.2,12.4,6.9,0.6,2.2,15.4,17.6,NaN,1.9,1.1,3.0,0.060,NaN,-1.5,-1.5,-3.0,-0.6,MIN,1995/1996,MIN
3,4,Doug West,27,71,2328,11.6,0.528,0.080,0.323,3.2,8.7,6.0,14.3,1.5,0.8,12.6,19.6,NaN,1.2,0.6,1.8,0.038,NaN,-2.2,-1.2,-3.4,-0.8,MIN,1995/1996,MIN
4,5,Winston Garland,30,73,1931,10.1,0.488,0.183,0.273,3.1,7.5,5.3,26.7,1.9,0.5,18.6,13.4,NaN,0.6,0.6,1.2,0.030,NaN,-2.2,-0.3,-2.6,-0.3,MIN,1995/1996,MIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12621,20,Jeenathan Williams,23,5,127,15.5,0.657,0.205,0.077,9.0,4.6,6.8,12.1,1.2,1.4,9.0,15.2,NaN,0.3,0.0,0.3,0.120,NaN,-1.0,-2.1,-3.0,0.0,POR,2023/2024,POR
12622,21,Shaquille Harrison,29,5,120,17.9,0.516,0.278,0.417,1.0,20.3,10.5,34.9,4.5,1.5,12.3,17.6,NaN,0.2,0.2,0.3,0.137,NaN,-2.5,2.6,0.1,0.1,POR,2023/2024,POR
12623,22,Greg Brown III,21,16,93,5.8,0.421,0.250,0.429,2.5,21.2,11.7,4.5,2.1,4.8,15.3,18.4,NaN,-0.3,0.1,-0.1,-0.074,NaN,-8.5,0.0,-8.5,-0.2,POR,2023/2024,POR
12624,23,Justin Minaya,23,4,89,4.0,0.363,0.522,0.043,3.8,15.6,9.7,5.9,1.1,5.0,14.6,13.4,NaN,-0.2,0.1,-0.1,-0.067,NaN,-7.2,-1.9,-9.0,-0.2,POR,2023/2024,POR


From advanced_df, I want "PER", "TS%", "3PAr", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "WS", "WS/48", "BPM", "OBPM", "VORP", "TRB%" to be added to our combined_df

"PER", "WS", "WS/48", "BPM" are very good and advanced measures of a players' overall individual performances because they compare to league averages and are leveled to be not affected by minutes played.

These measures are used a lot by NBA when choosing seasonal MVPs, therefore if I want to make a model to predict NBA players' improvement/performances, they will be very good target variables.
"OWS", "DWS" are related to "WS" which will be helpful.
"TS" is a good measure for accessing a players' shooting performance on court as it is a measure of shooting efficiency that takes into account of all field goals, 3-point field goals, and free throws. "3PAr" can be a helpful as 3-pointers have become more important in the NBA in recent years, we can use it to see a players shooting range.
"USG%" is very important in predicting a players' development and improvement because if a player is used more by their team in their early career, chances are they likely to improve faster. 

In [57]:
advanced_df_selected = advanced_df[["Player", "team", "season", "PER", "TS%", "3PAr", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "WS", "WS/48", "BPM", "OBPM", "VORP", "TRB%"]]
combined_df = pd.merge(combined_df, advanced_df_selected, on=['Player', 'team', 'season'], how='left') #Adding the wanted data from advanced DF to our final combined_df
combined_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,1,Isaiah Rider,23,75,67,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8
1,2,Christian Laettner,25,81,80,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6
2,3,Doug West,27,71,65,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0
3,4,Tom Gugliotta,25,31,17,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4
4,5,Sean Rooks,25,80,70,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12621,20,Ryan Arcidiacono,28,9,4,16.2,0.9,3.6,0.250,0.8,2.2,0.350,0.1,1.3,0.083,0.359,0.0,0.0,NaN,0.0,1.2,1.2,2.3,0.3,0.0,0.7,1.6,2.6,POR,2023/2024,PG,6-3,195,5,2.6,0.359,0.625,18.3,1.0,0.0,15.8,11.3,-0.2,-0.1,-0.049,-9.4,-7.0,-0.3,4.3
12622,21,John Butler,20,19,1,11.6,0.9,2.8,0.321,0.4,1.8,0.229,0.5,0.9,0.500,0.396,0.2,0.2,0.750,0.2,0.7,0.9,0.6,0.4,0.5,0.1,1.2,2.4,POR,2023/2024,C,7-1,175,0,5.7,0.411,0.660,6.5,1.5,3.6,1.8,11.0,-0.1,0.0,0.001,-6.2,-5.4,-0.2,4.4
12623,22,Jabari Walker,20,56,0,11.1,1.5,3.5,0.419,0.3,1.0,0.286,1.2,2.5,0.472,0.460,0.6,0.8,0.756,0.9,1.5,2.3,0.6,0.2,0.2,0.5,1.2,3.9,POR,2023/2024,SF,6-9,215,0,9.6,0.496,0.283,7.8,0.8,1.9,11.8,17.3,-0.1,0.2,0.015,-6.1,-3.9,-0.6,12.2
12624,23,Keon Johnson,20,40,0,10.4,1.7,4.5,0.376,0.7,2.0,0.346,1.0,2.4,0.402,0.455,0.7,1.0,0.659,0.3,0.8,1.1,1.5,0.5,0.2,1.1,1.1,4.7,POR,2023/2024,SG,6-5,186,1,9.0,0.482,0.455,20.9,2.2,1.3,18.0,25.0,-0.7,-0.5,-0.057,-5.4,-3.8,-0.4,6.1


Now I will import my data scraped from the website "basketball reference". For full scraping breakdown please look at the webscrapping.ipynb

In [58]:
com_df = pd.read_csv('com_df.csv')

In [59]:
tables0 = pd.read_html('https://www.basketball-reference.com/players/g/gilgesh01.html#all_per_game-playoffs_per_game')
shaip = tables0[1]
shaip["Player"] = "Shai Gilgeous-Alexander"
shaip["team"] = shaip["Tm"]
season_mapping = {
    '2018-19': '2019/2020',
    '2019-20': '2020/2021',
    '2020-21': '2021/2022',
    '2021-22': '2022/2023',
    '2022-23': '2023/2024',
    '2023-24': '2024/2025'
}
shaip['season'] = shaip['Season'].replace(season_mapping)
shaip = shaip.iloc[:-4]
shaip = shaip[shaip['season'] != '2019/2020']
shaip.drop(["Tm", "Lg", "Awards", "Season"], axis = 1, inplace = True)
shaip

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season
1,21.0,SG,70.0,70.0,34.7,6.8,14.5,0.471,1.2,3.6,0.347,5.6,10.9,0.512,0.514,4.1,5.1,0.807,0.7,5.2,5.9,3.3,1.1,0.7,1.9,1.7,19.0,Shai Gilgeous-Alexander,OKC,2020/2021
2,22.0,SG,35.0,35.0,33.7,8.2,16.1,0.508,2.0,4.9,0.418,6.2,11.3,0.547,0.571,5.3,6.5,0.808,0.5,4.2,4.7,5.9,0.8,0.7,3.0,2.0,23.7,Shai Gilgeous-Alexander,OKC,2021/2022
3,23.0,PG,56.0,56.0,34.7,8.5,18.8,0.453,1.6,5.3,0.300,6.9,13.5,0.514,0.496,5.9,7.2,0.810,0.7,4.3,5.0,5.9,1.3,0.8,2.8,2.5,24.5,Shai Gilgeous-Alexander,OKC,2022/2023
4,24.0,PG,68.0,68.0,35.5,10.4,20.3,0.510,0.9,2.5,0.345,9.5,17.8,0.533,0.531,9.8,10.9,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,Shai Gilgeous-Alexander,OKC,2023/2024
5,25.0,PG,75.0,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,9.3,16.2,0.576,0.567,7.6,8.7,0.874,0.9,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,Shai Gilgeous-Alexander,OKC,2024/2025


In [60]:
tables1 = pd.read_html('https://www.basketball-reference.com/players/g/gilgesh01.html#all_per_game-playoffs_per_game')
shaia = tables0[6]
shaia["Player"] = "Shai Gilgeous-Alexander"
shaia["team"] = shaia["Tm"]
season_mapping = {
    '2018-19': '2019/2020',
    '2019-20': '2020/2021',
    '2020-21': '2021/2022',
    '2021-22': '2022/2023',
    '2022-23': '2023/2024',
    '2023-24': '2024/2025'
}
shaia['season'] = shaia['Season'].replace(season_mapping)
shaia = shaia.iloc[:-4]
shaia = shaia[shaia['season'] != '2019/2020']
shaia

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player,team,season
1,2019-20,21.0,OKC,NBA,SG,70.0,2428.0,17.7,0.568,0.247,0.352,2.2,16.1,9.4,15.2,1.6,1.7,10.3,23.7,NaN,3.1,3.0,6.1,0.121,NaN,1.3,0.4,1.7,2.2,Shai Gilgeous-Alexander,OKC,2020/2021
2,2020-21,22.0,OKC,NBA,SG,35.0,1180.0,21.6,0.623,0.301,0.405,1.7,13.1,7.3,31.2,1.1,1.7,13.7,27.8,NaN,2.7,0.8,3.5,0.143,NaN,3.9,0.2,4.1,1.8,Shai Gilgeous-Alexander,OKC,2021/2022
3,2021-22,23.0,OKC,NBA,PG,56.0,1942.0,20.9,0.557,0.282,0.385,1.9,13.0,7.4,31.1,1.8,2.2,11.1,30.8,NaN,2.8,1.8,4.6,0.113,NaN,2.9,0.5,3.4,2.7,Shai Gilgeous-Alexander,OKC,2022/2023
4,2022-23,24.0,OKC,NBA,PG,68.0,2416.0,27.2,0.626,0.122,0.535,2.6,12.2,7.3,25.7,2.2,2.5,10.1,32.8,NaN,8.4,3.0,11.4,0.226,NaN,5.7,1.5,7.3,5.6,Shai Gilgeous-Alexander,OKC,2023/2024
5,2023-24,25.0,OKC,NBA,PG,75.0,2553.0,29.3,0.636,0.181,0.436,3.0,14.7,9.1,29.8,2.8,2.4,8.4,32.8,NaN,10.5,4.2,14.6,0.275,NaN,6.7,2.3,9.0,7.1,Shai Gilgeous-Alexander,OKC,2024/2025


In [61]:
team0 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2024.html')
team0 = team0[0]
team0['season'] = '2024/2025'
team0['team'] = 'OKC'

team1 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2023.html')
team1 = team1[0]
team1['season'] = '2023/2024'
team1['team'] = 'OKC'

team2 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2022.html')
team2 = team2[0]
team2['season'] = '2022/2023'
team2['team'] = 'OKC'

team3 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2021.html')
team3 = team3[0]
team3['season'] = '2021/2022'
team3['team'] = 'OKC'

team4 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2020.html')
team4 = team4[0]
team4['season'] = '2020/2021'
team4['team'] = 'OKC'

shair = pd.concat([team0, team1, team2, team3, team4], ignore_index=True)
shair = shair.loc[shair["Player"] == "Shai Gilgeous-Alexander"]
shair

,No.,Player,Pos,Ht,Wt,Birth Date,Unnamed: 6,Exp,College,season,team
6,2,Shai Gilgeous-Alexander,PG,6-6,180,"July 12, 1998",ca,5,Kentucky,2024/2025,OKC
23,2,Shai Gilgeous-Alexander,PG,6-6,180,"July 12, 1998",ca,4,Kentucky,2023/2024,OKC
45,2,Shai Gilgeous-Alexander,PG,6-6,180,"July 12, 1998",ca,3,Kentucky,2022/2023,OKC
70,2,Shai Gilgeous-Alexander,SG,6-6,180,"July 12, 1998",ca,2,Kentucky,2021/2022,OKC
92,2,Shai Gilgeous-Alexander,SG,6-6,180,"July 12, 1998",ca,1,Kentucky,2020/2021,OKC


In [62]:
shair = shair[["Player", "season", "Ht", "Wt", "Exp"]] 
shai = pd.merge(shaip, shair, on=['Player', 'season'], how='inner') 
shai

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season,Ht,Wt,Exp
0,21.0,SG,70.0,70.0,34.7,6.8,14.5,0.471,1.2,3.6,0.347,5.6,10.9,0.512,0.514,4.1,5.1,0.807,0.7,5.2,5.9,3.3,1.1,0.7,1.9,1.7,19.0,Shai Gilgeous-Alexander,OKC,2020/2021,6-6,180,1
1,22.0,SG,35.0,35.0,33.7,8.2,16.1,0.508,2.0,4.9,0.418,6.2,11.3,0.547,0.571,5.3,6.5,0.808,0.5,4.2,4.7,5.9,0.8,0.7,3.0,2.0,23.7,Shai Gilgeous-Alexander,OKC,2021/2022,6-6,180,2
2,23.0,PG,56.0,56.0,34.7,8.5,18.8,0.453,1.6,5.3,0.300,6.9,13.5,0.514,0.496,5.9,7.2,0.810,0.7,4.3,5.0,5.9,1.3,0.8,2.8,2.5,24.5,Shai Gilgeous-Alexander,OKC,2022/2023,6-6,180,3
3,24.0,PG,68.0,68.0,35.5,10.4,20.3,0.510,0.9,2.5,0.345,9.5,17.8,0.533,0.531,9.8,10.9,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,Shai Gilgeous-Alexander,OKC,2023/2024,6-6,180,4
4,25.0,PG,75.0,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,9.3,16.2,0.576,0.567,7.6,8.7,0.874,0.9,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,Shai Gilgeous-Alexander,OKC,2024/2025,6-6,180,5


In [63]:
shaia = shaia[["Player", "team", "season", "PER", "TS%", "3PAr", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "WS", "WS/48", "BPM", "OBPM", "VORP", "TRB%"]]
shai = pd.merge(shai, shaia, on=['Player', 'team', 'season'], how='left')
shai

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,21.0,SG,70.0,70.0,34.7,6.8,14.5,0.471,1.2,3.6,0.347,5.6,10.9,0.512,0.514,4.1,5.1,0.807,0.7,5.2,5.9,3.3,1.1,0.7,1.9,1.7,19.0,Shai Gilgeous-Alexander,OKC,2020/2021,6-6,180,1,17.7,0.568,0.247,15.2,1.6,1.7,10.3,23.7,3.1,6.1,0.121,1.7,1.3,2.2,9.4
1,22.0,SG,35.0,35.0,33.7,8.2,16.1,0.508,2.0,4.9,0.418,6.2,11.3,0.547,0.571,5.3,6.5,0.808,0.5,4.2,4.7,5.9,0.8,0.7,3.0,2.0,23.7,Shai Gilgeous-Alexander,OKC,2021/2022,6-6,180,2,21.6,0.623,0.301,31.2,1.1,1.7,13.7,27.8,2.7,3.5,0.143,4.1,3.9,1.8,7.3
2,23.0,PG,56.0,56.0,34.7,8.5,18.8,0.453,1.6,5.3,0.300,6.9,13.5,0.514,0.496,5.9,7.2,0.810,0.7,4.3,5.0,5.9,1.3,0.8,2.8,2.5,24.5,Shai Gilgeous-Alexander,OKC,2022/2023,6-6,180,3,20.9,0.557,0.282,31.1,1.8,2.2,11.1,30.8,2.8,4.6,0.113,3.4,2.9,2.7,7.4
3,24.0,PG,68.0,68.0,35.5,10.4,20.3,0.510,0.9,2.5,0.345,9.5,17.8,0.533,0.531,9.8,10.9,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,Shai Gilgeous-Alexander,OKC,2023/2024,6-6,180,4,27.2,0.626,0.122,25.7,2.2,2.5,10.1,32.8,8.4,11.4,0.226,7.3,5.7,5.6,7.3
4,25.0,PG,75.0,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,9.3,16.2,0.576,0.567,7.6,8.7,0.874,0.9,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,Shai Gilgeous-Alexander,OKC,2024/2025,6-6,180,5,29.3,0.636,0.181,29.8,2.8,2.4,8.4,32.8,10.5,14.6,0.275,9.0,6.7,7.1,9.1


In [64]:
shai

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,21.0,SG,70.0,70.0,34.7,6.8,14.5,0.471,1.2,3.6,0.347,5.6,10.9,0.512,0.514,4.1,5.1,0.807,0.7,5.2,5.9,3.3,1.1,0.7,1.9,1.7,19.0,Shai Gilgeous-Alexander,OKC,2020/2021,6-6,180,1,17.7,0.568,0.247,15.2,1.6,1.7,10.3,23.7,3.1,6.1,0.121,1.7,1.3,2.2,9.4
1,22.0,SG,35.0,35.0,33.7,8.2,16.1,0.508,2.0,4.9,0.418,6.2,11.3,0.547,0.571,5.3,6.5,0.808,0.5,4.2,4.7,5.9,0.8,0.7,3.0,2.0,23.7,Shai Gilgeous-Alexander,OKC,2021/2022,6-6,180,2,21.6,0.623,0.301,31.2,1.1,1.7,13.7,27.8,2.7,3.5,0.143,4.1,3.9,1.8,7.3
2,23.0,PG,56.0,56.0,34.7,8.5,18.8,0.453,1.6,5.3,0.300,6.9,13.5,0.514,0.496,5.9,7.2,0.810,0.7,4.3,5.0,5.9,1.3,0.8,2.8,2.5,24.5,Shai Gilgeous-Alexander,OKC,2022/2023,6-6,180,3,20.9,0.557,0.282,31.1,1.8,2.2,11.1,30.8,2.8,4.6,0.113,3.4,2.9,2.7,7.4
3,24.0,PG,68.0,68.0,35.5,10.4,20.3,0.510,0.9,2.5,0.345,9.5,17.8,0.533,0.531,9.8,10.9,0.905,0.9,4.0,4.8,5.5,1.6,1.0,2.8,2.8,31.4,Shai Gilgeous-Alexander,OKC,2023/2024,6-6,180,4,27.2,0.626,0.122,25.7,2.2,2.5,10.1,32.8,8.4,11.4,0.226,7.3,5.7,5.6,7.3
4,25.0,PG,75.0,75.0,34.0,10.6,19.8,0.535,1.3,3.6,0.353,9.3,16.2,0.576,0.567,7.6,8.7,0.874,0.9,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,Shai Gilgeous-Alexander,OKC,2024/2025,6-6,180,5,29.3,0.636,0.181,29.8,2.8,2.4,8.4,32.8,10.5,14.6,0.275,9.0,6.7,7.1,9.1


In [65]:
tables0 = pd.read_html('https://www.basketball-reference.com/players/d/duranke01.html#all_per_game-playoffs_per_game')
kp = tables0[1]
kp["Player"] = "Kevin Durant"
kp["team"] = kp["Tm"]
season_mapping = {
    '2007-08': '2008/2009',
    '2008-09': '2009/2010',
    '2009-10': '2010/2011',
    '2010-11': '2011/2012',
    '2011-12': '2012/2013',
    '2012-13': '2013/2014',
    '2013-14': '2014/2015',
    '2014-15': '2015/2016',
    '2015-16': '2016/2017',
    '2016-17': '2017/2018',
    '2017-18': '2018/2019',
    '2018-19': '2019/2020',
    '2019-20': '2020/2021',
    '2020-21': '2021/2022',
    '2021-22': '2022/2023',
    '2022-23': '2023/2024',
    '2023-24': '2024/2025'
}
kp['season'] = kp['Season'].replace(season_mapping)
kp = kp.iloc[:-6]
kp = kp[kp['season'] != '2020/2021']
kp.drop(["Tm", "Lg", "Awards", "Season"], axis = 1, inplace = True)
kp

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season
0,19.0,SG,80,80,34.6,7.3,17.1,.430,0.7,2.6,.288,6.6,14.5,.455,.451,4.9,5.6,.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3,Kevin Durant,SEA,2008/2009
1,20.0,SF,74,74,39.0,8.9,18.8,.476,1.3,3.1,.422,7.6,15.7,.486,.510,6.1,7.1,.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3,Kevin Durant,OKC,2009/2010
2,21.0,SF,82,82,39.5,9.7,20.3,.476,1.6,4.3,.365,8.1,16.1,.506,.514,9.2,10.2,.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1,Kevin Durant,OKC,2010/2011
3,22.0,SF,78,78,38.9,9.1,19.7,.462,1.9,5.3,.350,7.3,14.4,.504,.509,7.6,8.7,.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7,Kevin Durant,OKC,2011/2012
4,23.0,SF,66,66,38.6,9.7,19.7,.496,2.0,5.2,.387,7.7,14.4,.535,.547,6.5,7.6,.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0,Kevin Durant,OKC,2012/2013
5,24.0,SF,81,81,38.5,9.0,17.7,.510,1.7,4.1,.416,7.3,13.6,.539,.559,8.4,9.3,.905,0.6,7.3,7.9,4.6,1.4,1.3,3.5,1.8,28.1,Kevin Durant,OKC,2013/2014
6,25.0,SF,81,81,38.5,10.5,20.8,.503,2.4,6.1,.391,8.1,14.8,.549,.560,8.7,9.9,.873,0.7,6.7,7.4,5.5,1.3,0.7,3.5,2.1,32.0,Kevin Durant,OKC,2014/2015
7,26.0,SF,27,27,33.8,8.8,17.3,.510,2.4,5.9,.403,6.4,11.4,.565,.578,5.4,6.3,.854,0.6,6.0,6.6,4.1,0.9,0.9,2.7,1.5,25.4,Kevin Durant,OKC,2015/2016
8,27.0,SF,72,72,35.8,9.7,19.2,.505,2.6,6.7,.387,7.1,12.5,.569,.573,6.2,6.9,.898,0.6,7.6,8.2,5.0,1.0,1.2,3.5,1.9,28.2,Kevin Durant,OKC,2016/2017
9,28.0,PF,62,62,33.4,8.9,16.5,.537,1.9,5.0,.375,7.0,11.5,.608,.594,5.4,6.2,.875,0.6,7.6,8.3,4.8,1.1,1.6,2.2,1.9,25.1,Kevin Durant,GSW,2017/2018


In [66]:
tables1 = pd.read_html('https://www.basketball-reference.com/players/d/duranke01.html#all_per_game-playoffs_per_game')
ka = tables0[6]
ka["Player"] = "Kevin Durant"
ka["team"] = ka["Tm"]
season_mapping = {
    '2007-08': '2008/2009',
    '2008-09': '2009/2010',
    '2009-10': '2010/2011',
    '2010-11': '2011/2012',
    '2011-12': '2012/2013',
    '2012-13': '2013/2014',
    '2013-14': '2014/2015',
    '2014-15': '2015/2016',
    '2015-16': '2016/2017',
    '2016-17': '2017/2018',
    '2017-18': '2018/2019',
    '2018-19': '2019/2020',
    '2019-20': '2020/2021',
    '2020-21': '2021/2022',
    '2021-22': '2022/2023',
    '2022-23': '2023/2024',
    '2023-24': '2024/2025'
}

ka['season'] = ka['Season'].replace(season_mapping)
ka = ka.iloc[:-6]
ka

,Season,Age,Tm,Lg,Pos,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Player,team,season
0,2007-08,19.0,SEA,NBA,SG,80.0,2768.0,15.8,0.519,0.150,0.328,2.8,10.9,6.9,12.0,1.4,1.9,12.9,28.1,NaN,0.4,1.9,2.3,0.040,NaN,0.5,-0.7,-0.1,1.3,Kevin Durant,SEA,2008/2009
1,2008-09,20.0,OKC,NBA,SF,74.0,2885.0,20.8,0.577,0.165,0.377,3.0,16.4,9.6,13.5,1.7,1.4,12.2,28.3,NaN,5.2,2.7,7.9,0.132,NaN,3.4,-0.1,3.3,3.8,Kevin Durant,OKC,2009/2010
2,2009-10,21.0,OKC,NBA,SF,82.0,3239.0,26.2,0.607,0.210,0.504,3.8,17.9,11.0,13.5,1.8,1.9,11.7,32.0,NaN,11.1,5.0,16.1,0.238,NaN,6.3,0.8,7.1,7.5,Kevin Durant,OKC,2010/2011
3,2010-11,22.0,OKC,NBA,SF,78.0,3038.0,23.6,0.589,0.269,0.439,2.3,17.7,10.3,13.2,1.5,1.8,10.6,30.6,NaN,8.6,3.3,12.0,0.189,NaN,5.2,-0.3,4.9,5.3,Kevin Durant,OKC,2011/2012
4,2011-12,23.0,OKC,NBA,SF,66.0,2546.0,26.2,0.610,0.265,0.386,1.9,20.4,11.8,17.5,1.8,2.2,14.0,31.3,NaN,8.5,3.7,12.2,0.230,NaN,6.2,0.8,7.0,5.8,Kevin Durant,OKC,2012/2013
5,2012-13,24.0,OKC,NBA,SF,81.0,3119.0,28.3,0.647,0.233,0.523,1.8,20.3,11.8,21.7,1.9,2.6,13.7,29.8,NaN,13.6,5.3,18.9,0.291,NaN,7.6,1.7,9.3,8.9,Kevin Durant,OKC,2013/2014
6,2013-14,25.0,OKC,NBA,SF,81.0,3122.0,29.8,0.635,0.291,0.477,2.2,18.6,10.8,26.7,1.7,1.5,12.2,33.0,NaN,14.8,4.4,19.2,0.295,NaN,8.8,1.4,10.2,9.6,Kevin Durant,OKC,2014/2015
7,2014-15,26.0,OKC,NBA,SF,27.0,913.0,27.6,0.633,0.340,0.366,1.9,18.7,10.4,22.2,1.3,2.2,12.0,29.1,NaN,3.8,1.0,4.8,0.252,NaN,8.6,1.4,10.0,2.8,Kevin Durant,OKC,2015/2016
8,2015-16,27.0,OKC,NBA,SF,72.0,2578.0,28.2,0.634,0.348,0.361,2.0,21.8,12.4,24.2,1.3,2.5,13.5,30.6,NaN,11.0,3.5,14.5,0.270,NaN,8.4,1.6,9.9,7.8,Kevin Durant,OKC,2016/2017
9,2016-17,28.0,GSW,NBA,PF,62.0,2070.0,27.6,0.651,0.304,0.374,2.2,23.6,13.6,23.1,1.5,3.8,10.4,27.8,NaN,8.0,4.0,12.0,0.278,NaN,6.7,2.2,8.9,5.7,Kevin Durant,GSW,2017/2018


In [68]:

team0 = pd.read_html('https://www.basketball-reference.com/teams/SEA/2008.html')
team0 = team0[0]
team0['season'] = '2008/2009'
team0['team'] = 'SEA'

team1 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2009.html')
team1 = team1[0]
team1['season'] = '2009/2010'
team1['team'] = 'OKC'

team2 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2010.html')
team2 = team2[0]
team2['season'] = '2010/2011'
team2['team'] = 'OKC'

team3 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2011.html')
team3 = team3[0]
team3['season'] = '2011/2012'
team3['team'] = 'OKC'

team4 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2012.html')
team4 = team4[0]
team4['season'] = '2012/2013'
team4['team'] = 'OKC'

team5 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2013.html')
team5 = team5[0]
team5['season'] = '2013/2014'
team5['team'] = 'OKC'

team6 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2014.html')
team6 = team6[0]
team6['season'] = '2014/2015'
team6['team'] = 'OKC'

team7 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2015.html')
team7 = team7[0]
team7['season'] = '2015/2016'
team7['team'] = 'OKC'

team8 = pd.read_html('https://www.basketball-reference.com/teams/OKC/2016.html')
team8 = team8[0]
team8['season'] = '2016/2017'
team8['team'] = 'OKC'

team9 = pd.read_html('https://www.basketball-reference.com/teams/GSW/2017.html')
team9 = team9[0]
team9['season'] = '2017/2018'
team9['team'] = 'GSW'

team10 = pd.read_html('https://www.basketball-reference.com/teams/GSW/2018.html')
team10 = team10[0]
team10['season'] = '2018/2019'
team10['team'] = 'GSW'

team11 = pd.read_html('https://www.basketball-reference.com/teams/GSW/2019.html')
team11 = team11[0]
team11['season'] = '2019/2020'
team11['team'] = 'GSW'

team12 = pd.read_html('https://www.basketball-reference.com/teams/BRK/2020.html')
team12 = team12[0]
team12['season'] = '2020/2021'
team12['team'] = 'BRK'

team13 = pd.read_html('https://www.basketball-reference.com/teams/BRK/2021.html')
team13 = team13[0]
team13['season'] = '2021/2022'
team13['team'] = 'BRK'

team14 = pd.read_html('https://www.basketball-reference.com/teams/BRK/2022.html')
team14 = team14[0]
team14['season'] = '2022/2023'
team14['team'] = 'BRK'

team15 = pd.read_html('https://www.basketball-reference.com/teams/PHO/2023.html')
team15 = team15[0]
team15['season'] = '2023/2024'
team15['team'] = 'PHO'

team16 = pd.read_html('https://www.basketball-reference.com/teams/PHO/2024.html')
team16 = team16[0]
team16['season'] = '2024/2025'
team16['team'] = 'PHO'

kr = pd.concat([team0, team1, team2, team3, team4, team5, team6, team7, team8, team9, team10, team11, team12, team13, team14, team15, team16,], ignore_index=True)
kr = kr.loc[kr["Player"] == "Kevin Durant"]
kr

,No.,Player,Pos,Ht,Wt,Birth Date,Unnamed: 6,Exp,College,season,team
2,35,Kevin Durant,SG,6-11,240,"September 29, 1988",us,R,Texas,2008/2009,SEA
23,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,1,Texas,2009/2010,OKC
44,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,2,Texas,2010/2011,OKC
62,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,3,Texas,2011/2012,OKC
80,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,4,Texas,2012/2013,OKC
95,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,5,Texas,2013/2014,OKC
112,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,6,Texas,2014/2015,OKC
130,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,7,Texas,2015/2016,OKC
151,35,Kevin Durant,SF,6-11,240,"September 29, 1988",us,8,Texas,2016/2017,OKC
168,35,Kevin Durant,PF,6-11,240,"September 29, 1988",us,9,Texas,2017/2018,GSW


In [69]:
kr = kr[["Player", "season", "Ht", "Wt", "Exp"]] 
k = pd.merge(kp, kr, on=['Player', 'season'], how='inner') 
k

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season,Ht,Wt,Exp
0,19.0,SG,80,80,34.6,7.3,17.1,.430,0.7,2.6,.288,6.6,14.5,.455,.451,4.9,5.6,.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3,Kevin Durant,SEA,2008/2009,6-11,240,R
1,20.0,SF,74,74,39.0,8.9,18.8,.476,1.3,3.1,.422,7.6,15.7,.486,.510,6.1,7.1,.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3,Kevin Durant,OKC,2009/2010,6-11,240,1
2,21.0,SF,82,82,39.5,9.7,20.3,.476,1.6,4.3,.365,8.1,16.1,.506,.514,9.2,10.2,.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1,Kevin Durant,OKC,2010/2011,6-11,240,2
3,22.0,SF,78,78,38.9,9.1,19.7,.462,1.9,5.3,.350,7.3,14.4,.504,.509,7.6,8.7,.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7,Kevin Durant,OKC,2011/2012,6-11,240,3
4,23.0,SF,66,66,38.6,9.7,19.7,.496,2.0,5.2,.387,7.7,14.4,.535,.547,6.5,7.6,.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0,Kevin Durant,OKC,2012/2013,6-11,240,4
5,24.0,SF,81,81,38.5,9.0,17.7,.510,1.7,4.1,.416,7.3,13.6,.539,.559,8.4,9.3,.905,0.6,7.3,7.9,4.6,1.4,1.3,3.5,1.8,28.1,Kevin Durant,OKC,2013/2014,6-11,240,5
6,25.0,SF,81,81,38.5,10.5,20.8,.503,2.4,6.1,.391,8.1,14.8,.549,.560,8.7,9.9,.873,0.7,6.7,7.4,5.5,1.3,0.7,3.5,2.1,32.0,Kevin Durant,OKC,2014/2015,6-11,240,6
7,26.0,SF,27,27,33.8,8.8,17.3,.510,2.4,5.9,.403,6.4,11.4,.565,.578,5.4,6.3,.854,0.6,6.0,6.6,4.1,0.9,0.9,2.7,1.5,25.4,Kevin Durant,OKC,2015/2016,6-11,240,7
8,27.0,SF,72,72,35.8,9.7,19.2,.505,2.6,6.7,.387,7.1,12.5,.569,.573,6.2,6.9,.898,0.6,7.6,8.2,5.0,1.0,1.2,3.5,1.9,28.2,Kevin Durant,OKC,2016/2017,6-11,240,8
9,28.0,PF,62,62,33.4,8.9,16.5,.537,1.9,5.0,.375,7.0,11.5,.608,.594,5.4,6.2,.875,0.6,7.6,8.3,4.8,1.1,1.6,2.2,1.9,25.1,Kevin Durant,GSW,2017/2018,6-11,240,9


In [70]:
ka = ka[["Player", "team", "season", "PER", "TS%", "3PAr", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "WS", "WS/48", "BPM", "OBPM", "VORP", "TRB%"]]
k = pd.merge(k, ka, on=['Player', 'team', 'season'], how='left')
k

,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player,team,season,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,19.0,SG,80,80,34.6,7.3,17.1,.430,0.7,2.6,.288,6.6,14.5,.455,.451,4.9,5.6,.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3,Kevin Durant,SEA,2008/2009,6-11,240,R,15.8,0.519,0.150,12.0,1.4,1.9,12.9,28.1,0.4,2.3,0.040,-0.1,0.5,1.3,6.9
1,20.0,SF,74,74,39.0,8.9,18.8,.476,1.3,3.1,.422,7.6,15.7,.486,.510,6.1,7.1,.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3,Kevin Durant,OKC,2009/2010,6-11,240,1,20.8,0.577,0.165,13.5,1.7,1.4,12.2,28.3,5.2,7.9,0.132,3.3,3.4,3.8,9.6
2,21.0,SF,82,82,39.5,9.7,20.3,.476,1.6,4.3,.365,8.1,16.1,.506,.514,9.2,10.2,.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1,Kevin Durant,OKC,2010/2011,6-11,240,2,26.2,0.607,0.210,13.5,1.8,1.9,11.7,32.0,11.1,16.1,0.238,7.1,6.3,7.5,11.0
3,22.0,SF,78,78,38.9,9.1,19.7,.462,1.9,5.3,.350,7.3,14.4,.504,.509,7.6,8.7,.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7,Kevin Durant,OKC,2011/2012,6-11,240,3,23.6,0.589,0.269,13.2,1.5,1.8,10.6,30.6,8.6,12.0,0.189,4.9,5.2,5.3,10.3
4,23.0,SF,66,66,38.6,9.7,19.7,.496,2.0,5.2,.387,7.7,14.4,.535,.547,6.5,7.6,.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0,Kevin Durant,OKC,2012/2013,6-11,240,4,26.2,0.610,0.265,17.5,1.8,2.2,14.0,31.3,8.5,12.2,0.230,7.0,6.2,5.8,11.8
5,24.0,SF,81,81,38.5,9.0,17.7,.510,1.7,4.1,.416,7.3,13.6,.539,.559,8.4,9.3,.905,0.6,7.3,7.9,4.6,1.4,1.3,3.5,1.8,28.1,Kevin Durant,OKC,2013/2014,6-11,240,5,28.3,0.647,0.233,21.7,1.9,2.6,13.7,29.8,13.6,18.9,0.291,9.3,7.6,8.9,11.8
6,25.0,SF,81,81,38.5,10.5,20.8,.503,2.4,6.1,.391,8.1,14.8,.549,.560,8.7,9.9,.873,0.7,6.7,7.4,5.5,1.3,0.7,3.5,2.1,32.0,Kevin Durant,OKC,2014/2015,6-11,240,6,29.8,0.635,0.291,26.7,1.7,1.5,12.2,33.0,14.8,19.2,0.295,10.2,8.8,9.6,10.8
7,26.0,SF,27,27,33.8,8.8,17.3,.510,2.4,5.9,.403,6.4,11.4,.565,.578,5.4,6.3,.854,0.6,6.0,6.6,4.1,0.9,0.9,2.7,1.5,25.4,Kevin Durant,OKC,2015/2016,6-11,240,7,27.6,0.633,0.340,22.2,1.3,2.2,12.0,29.1,3.8,4.8,0.252,10.0,8.6,2.8,10.4
8,27.0,SF,72,72,35.8,9.7,19.2,.505,2.6,6.7,.387,7.1,12.5,.569,.573,6.2,6.9,.898,0.6,7.6,8.2,5.0,1.0,1.2,3.5,1.9,28.2,Kevin Durant,OKC,2016/2017,6-11,240,8,28.2,0.634,0.348,24.2,1.3,2.5,13.5,30.6,11.0,14.5,0.270,9.9,8.4,7.8,12.4
9,28.0,PF,62,62,33.4,8.9,16.5,.537,1.9,5.0,.375,7.0,11.5,.608,.594,5.4,6.2,.875,0.6,7.6,8.3,4.8,1.1,1.6,2.2,1.9,25.1,Kevin Durant,GSW,2017/2018,6-11,240,9,27.6,0.651,0.304,23.1,1.5,3.8,10.4,27.8,8.0,12.0,0.278,8.9,6.7,5.7,13.6


In [ ]:
k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 48 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     18 non-null     float64
 1   Pos     18 non-null     object 
 2   G       18 non-null     object 
 3   GS      18 non-null     object 
 4   MP      18 non-null     object 
 5   FG      18 non-null     object 
 6   FGA     18 non-null     object 
 7   FG%     18 non-null     object 
 8   3P      18 non-null     object 
 9   3PA     18 non-null     object 
 10  3P%     18 non-null     object 
 11  2P      18 non-null     object 
 12  2PA     18 non-null     object 
 13  2P%     18 non-null     object 
 14  eFG%    18 non-null     object 
 15  FT      18 non-null     object 
 16  FTA     18 non-null     object 
 17  FT%     18 non-null     object 
 18  ORB     18 non-null     object 
 19  DRB     18 non-null     object 
 20  TRB     18 non-null     object 
 21  AST     18 non-null     object 
 22  STL 

In [71]:
# I have made sure com_df has the same columns as my combined_df, so all I have to do here is just to concat them

combined_df = pd.concat([combined_df, com_df, shai, k], ignore_index=True)
combined_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,1.0,Isaiah Rider,23.0,75,67,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8
1,2.0,Christian Laettner,25.0,81,80,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6
2,3.0,Doug West,27.0,71,65,32.8,4.9,10.7,0.461,0.2,0.9,0.18,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0
3,4.0,Tom Gugliotta,25.0,31,17,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4
4,5.0,Sean Rooks,25.0,80,70,30.1,3.6,7.7,0.47,0.0,0.1,0.0,3.6,7.6,0.474,0.47,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17194,NaN,Kevin Durant,33.0,55,55,37.2,10.5,20.3,.518,2.1,5.5,.383,8.4,14.8,.568,.570,6.8,7.4,.910,0.5,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8
17195,NaN,Kevin Durant,34.0,47,47,35.6,10.3,18.3,.560,2.0,4.9,.404,8.3,13.4,.617,.614,6.5,7.1,.919,0.4,6.3,6.7,5.0,0.7,1.4,3.3,2.1,29.1,TOT,2023/2024,PF,6-11,240,14,25.9,0.677,0.267,24.5,1.0,3.4,13.4,30.7,4.7,6.8,0.194,7.1,6.0,3.9,10.5
17196,NaN,Kevin Durant,34.0,39,39,36.0,10.5,18.8,.559,1.8,4.8,.376,8.7,14.0,.622,.607,6.8,7.3,.934,0.4,6.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5
17197,NaN,Kevin Durant,34.0,8,8,33.6,9.1,16.0,.570,2.8,5.1,.537,6.4,10.9,.586,.656,5.0,6.0,.833,0.4,6.0,6.4,3.5,0.3,1.3,2.5,0.9,26.0,PHO,2023/2024,PF,6-11,240,14,24.3,0.697,0.320,17.3,0.4,3.3,11.8,26.8,0.8,1.1,0.196,8.0,7.1,0.7,10.5


In [74]:
indices_to_drop = [938, 954, 970, 3433, 12103]
combined_df.drop(indices_to_drop, inplace=True)

combined_df.reset_index(drop=True, inplace=True)


In [76]:
combined_df = combined_df[combined_df['team'] != 'TOT']

In [85]:
columns_to_exclude = ['Player', 'team', 'season', 'Pos', 'Ht']
for column in combined_df.columns:
    if column not in columns_to_exclude:
        combined_df[column] = pd.to_numeric(combined_df[column])

C:\Users\ivant\AppData\Local\Temp\ipykernel_3152\2890827934.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df[column] = pd.to_numeric(combined_df[column])


In [79]:
# Some of the values in the "Exp" column is R which represents rookie, so I need to change them to 0 for numerical values.

combined_df['Exp'] = combined_df['Exp'].replace('R', 0)
combined_df['Exp'] = pd.to_numeric(combined_df['Exp'])

C:\Users\ivant\AppData\Local\Temp\ipykernel_3152\2051487927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Exp'] = combined_df['Exp'].replace('R', 0)
C:\Users\ivant\AppData\Local\Temp\ipykernel_3152\2051487927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Exp'] = pd.to_numeric(combined_df['Exp'])


# Rows and Columns dropping

There are 82 games in total per season, when the games played are minimal the stats can be misleading as the stats in our dataset are stats per game. Therefore I need to drop rows of players who plaed really minimal games. In Sprint 1 I dropped all the rows with players who didn't play at least 40 games in that season, but now I decided to change it to minimum games > 20, to preserve more data for modelling later on.

In [88]:
# Dropping all the rows with players who didn't play at least 20 games in that season, 

combined_df = combined_df[combined_df['G'] >= 20]
combined_df

,Rk,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,1.0,Isaiah Rider,23.0,75.0,67.0,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8
1,2.0,Christian Laettner,25.0,81.0,80.0,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6
2,3.0,Doug West,27.0,71.0,65.0,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0
3,4.0,Tom Gugliotta,25.0,31.0,17.0,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4
4,5.0,Sean Rooks,25.0,80.0,70.0,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17187,NaN,Kevin Durant,30.0,78.0,78.0,34.6,9.2,17.7,0.521,1.8,5.0,0.353,7.5,12.8,0.587,0.571,5.7,6.5,0.885,0.4,5.9,6.4,5.9,0.7,1.1,2.9,2.0,26.0,GSW,2019/2020,SF,6-11,240,11,24.2,0.631,0.281,26.2,1.0,2.6,12.3,29.0,8.6,11.5,0.204,5.5,5.4,5.1,9.8
17188,NaN,Kevin Durant,32.0,35.0,32.0,33.1,9.3,17.2,0.537,2.4,5.4,0.450,6.8,11.8,0.577,0.608,6.0,6.8,0.882,0.4,6.7,7.1,5.6,0.7,1.3,3.4,2.0,26.9,BRK,2021/2022,PF,6-11,240,12,26.4,0.666,0.313,27.5,1.0,3.4,14.5,31.2,3.7,5.0,0.206,7.2,6.4,2.7,11.8
17189,NaN,Kevin Durant,33.0,55.0,55.0,37.2,10.5,20.3,0.518,2.1,5.5,0.383,8.4,14.8,0.568,0.570,6.8,7.4,0.910,0.5,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8
17191,NaN,Kevin Durant,34.0,39.0,39.0,36.0,10.5,18.8,0.559,1.8,4.8,0.376,8.7,14.0,0.622,0.607,6.8,7.3,0.934,0.4,6.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5


In [89]:
# There is an extra column at the start, therefore we will remove a column to make one index column
combined_df = combined_df.drop(columns=['Rk'])
combined_df.head()
print(combined_df.shape)

(13521, 48)


In [90]:
combined_df = combined_df.reset_index(drop=True)
combined_df

,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%
0,Isaiah Rider,23.0,75.0,67.0,35.3,7.4,16.7,0.447,1.9,5.3,0.351,5.6,11.4,0.491,0.502,3.7,4.5,0.817,1.2,2.1,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8
1,Christian Laettner,25.0,81.0,80.0,34.2,5.6,11.4,0.489,0.2,0.5,0.325,5.4,10.9,0.497,0.496,5.0,6.2,0.818,2.0,5.5,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6
2,Doug West,27.0,71.0,65.0,32.8,4.9,10.7,0.461,0.2,0.9,0.180,4.8,9.9,0.485,0.468,2.9,3.5,0.837,0.8,2.4,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0
3,Tom Gugliotta,25.0,31.0,17.0,32.8,5.2,11.5,0.454,0.9,2.8,0.318,4.3,8.7,0.498,0.493,3.0,3.9,0.762,1.6,5.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4
4,Sean Rooks,25.0,80.0,70.0,30.1,3.6,7.7,0.470,0.0,0.1,0.000,3.6,7.6,0.474,0.470,3.6,4.8,0.761,2.1,4.0,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13516,Kevin Durant,30.0,78.0,78.0,34.6,9.2,17.7,0.521,1.8,5.0,0.353,7.5,12.8,0.587,0.571,5.7,6.5,0.885,0.4,5.9,6.4,5.9,0.7,1.1,2.9,2.0,26.0,GSW,2019/2020,SF,6-11,240,11,24.2,0.631,0.281,26.2,1.0,2.6,12.3,29.0,8.6,11.5,0.204,5.5,5.4,5.1,9.8
13517,Kevin Durant,32.0,35.0,32.0,33.1,9.3,17.2,0.537,2.4,5.4,0.450,6.8,11.8,0.577,0.608,6.0,6.8,0.882,0.4,6.7,7.1,5.6,0.7,1.3,3.4,2.0,26.9,BRK,2021/2022,PF,6-11,240,12,26.4,0.666,0.313,27.5,1.0,3.4,14.5,31.2,3.7,5.0,0.206,7.2,6.4,2.7,11.8
13518,Kevin Durant,33.0,55.0,55.0,37.2,10.5,20.3,0.518,2.1,5.5,0.383,8.4,14.8,0.568,0.570,6.8,7.4,0.910,0.5,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8
13519,Kevin Durant,34.0,39.0,39.0,36.0,10.5,18.8,0.559,1.8,4.8,0.376,8.7,14.0,0.622,0.607,6.8,7.3,0.934,0.4,6.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5


I have decided to drop FGA, FG, DRB, 3P, 2P, FT columns because of multi colinearity. We can get 3P from 3PA * 3P%, and so on.

In [91]:
combined_df.drop(['FGA', 'FG', 'DRB', '3P', '2P', 'FT'], axis=1, inplace=True)

# Data Cleaning

In [92]:
#Data cleaning. I will now check for missing/null values in our new dataframe.
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13521 entries, 0 to 13520
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  13521 non-null  object 
 1   Age     13521 non-null  float64
 2   G       13521 non-null  float64
 3   GS      13521 non-null  float64
 4   MP      13521 non-null  float64
 5   FG%     13521 non-null  float64
 6   3PA     13521 non-null  float64
 7   3P%     12547 non-null  float64
 8   2PA     13521 non-null  float64
 9   2P%     13521 non-null  float64
 10  eFG%    13521 non-null  float64
 11  FTA     13521 non-null  float64
 12  FT%     13509 non-null  float64
 13  ORB     13521 non-null  float64
 14  TRB     13521 non-null  float64
 15  AST     13521 non-null  float64
 16  STL     13521 non-null  float64
 17  BLK     13521 non-null  float64
 18  TOV     13521 non-null  float64
 19  PF      13521 non-null  float64
 20  PTS     13521 non-null  float64
 21  team    13521 non-null  object 
 22

Our data has a mixture of numerical and non-numerical columns, with most of them being floats. We have 5 integer data types which are "Age", "G", "GS" and "Wt", "Exp". 5 object data types which are "Player", "team", "season", "Pos", "Ht". Interestingly "Ht" is also an object even tho they are heights, we might need to change it later if we need to investigate it as number.

We only have 2 columns with missing data (<7452 non-null values). These are:

10  3P%     6852 non-null   float64  (A large portion)
17  FT%     7450 non-null   float64


In [93]:
# Checking for duplicates and counting
combined_df.duplicated().sum()

0

There are no duplicated data, which is good.

In [94]:
combined_df.isna().sum()

Player      0
Age         0
G           0
GS          0
MP          0
FG%         0
3PA         0
3P%       974
2PA         0
2P%         0
eFG%        0
FTA         0
FT%        12
ORB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
team        0
season      0
Pos         0
Ht          0
Wt          0
Exp         0
PER         0
TS%         0
3PAr        0
AST%        0
STL%        0
BLK%        0
TOV%        0
USG%        0
OWS         0
WS          0
WS/48       0
BPM         0
OBPM        0
VORP        0
TRB%        0
dtype: int64

In [95]:
# Percentage of missing values in each column
combined_df.isna().sum()/combined_df.shape[0]*100.0

Player    0.000000
Age       0.000000
G         0.000000
GS        0.000000
MP        0.000000
FG%       0.000000
3PA       0.000000
3P%       7.203609
2PA       0.000000
2P%       0.000000
eFG%      0.000000
FTA       0.000000
FT%       0.088751
ORB       0.000000
TRB       0.000000
AST       0.000000
STL       0.000000
BLK       0.000000
TOV       0.000000
PF        0.000000
PTS       0.000000
team      0.000000
season    0.000000
Pos       0.000000
Ht        0.000000
Wt        0.000000
Exp       0.000000
PER       0.000000
TS%       0.000000
3PAr      0.000000
AST%      0.000000
STL%      0.000000
BLK%      0.000000
TOV%      0.000000
USG%      0.000000
OWS       0.000000
WS        0.000000
WS/48     0.000000
BPM       0.000000
OBPM      0.000000
VORP      0.000000
TRB%      0.000000
dtype: float64

Only 3P% is missing a significant amount of data, 7.67%. While FT% is missing only 0.09% of its data. Lets firstly look at the missing FT% as there are only 12 rows missing.
Lets look at the two rows that are missing.

In [96]:
missing_ft = combined_df[combined_df['FT%'].isna()]
print(missing_ft)

                Player   Age     G   GS    MP    FG%  3PA    3P%  2PA    2P%  \
1863       Luke Walton  31.0  21.0  0.0  14.2  0.353  0.8  0.438  1.7  0.314   
2017  Denzel Valentine  28.0  22.0  0.0   9.3  0.371  2.0  0.409  0.8  0.278   
2462    Alex Poythress  24.0  25.0  0.0   4.2  0.423  0.4  0.364  0.6  0.467   
3613     Nick Anderson  33.0  21.0  0.0   8.0  0.246  1.9  0.256  0.9  0.222   
3669      Erik Daniels  22.0  21.0  0.0   3.4  0.333  0.1  0.500  0.8  0.313   
4975       Erik Murphy  23.0  24.0  0.0   2.6  0.231  0.1  0.000  0.4  0.300   
5828      Damjan Rudež  30.0  45.0  0.0   7.0  0.352  1.4  0.313  0.5  0.458   
5975     David Wingate  35.0  20.0  0.0   4.6  0.438  0.0    NaN  0.8  0.438   
6714        Sam Hauser  24.0  26.0  0.0   6.1  0.460  1.7  0.432  0.2  0.667   
7773    Justin Holiday  33.0  28.0  0.0  14.7  0.384  3.1  0.345  1.4  0.474   
8079       Mike Miller  35.0  47.0  2.0   7.9  0.355  1.1  0.365  0.2  0.300   
8460    Cartier Martin  30.0  23.0  0.0 

We can see that the missing FT% data is dued to the fact that these two players didn't attempt any free throws at all these two seasons, so 0/0 = undefined so we got null values. To fix this, I will just set these two FT% to 0.


In [97]:
combined_df['FT%'] = combined_df['FT%'].fillna(0)
#check
combined_df.isna().sum()

Player      0
Age         0
G           0
GS          0
MP          0
FG%         0
3PA         0
3P%       974
2PA         0
2P%         0
eFG%        0
FTA         0
FT%         0
ORB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
team        0
season      0
Pos         0
Ht          0
Wt          0
Exp         0
PER         0
TS%         0
3PAr        0
AST%        0
STL%        0
BLK%        0
TOV%        0
USG%        0
OWS         0
WS          0
WS/48       0
BPM         0
OBPM        0
VORP        0
TRB%        0
dtype: int64

974 and 9.97% missing values are quite a lot so I want to investigate what exactly happened.

In [98]:
missing_3p = combined_df[combined_df['3P%'].isna()]
print(missing_3p)

                   Player   Age     G    GS    MP    FG%  3PA  3P%  2PA  \
13    Charles Shackleford  28.0  21.0   2.0  11.4  0.600  0.0  NaN  3.1   
29          Marques Bragg  25.0  53.0   0.0   7.0  0.450  0.0  NaN  2.3   
35           Dean Garrett  30.0  68.0  47.0  24.5  0.573  0.0  NaN  5.7   
39       Stojko Vranković  33.0  53.0  35.0  14.5  0.561  0.0  NaN  2.6   
52        Stanley Roberts  27.0  74.0  44.0  17.9  0.495  0.0  NaN  5.2   
...                   ...   ...   ...   ...   ...    ...  ...  ...  ...   
9641          Chris Kaman  32.0  74.0  13.0  18.9  0.515  0.0  NaN  7.4   
9645        Joel Freeland  27.0  48.0   8.0  12.9  0.490  0.0  NaN  3.1   
9647      Thomas Robinson  23.0  32.0   4.0  12.2  0.516  0.0  NaN  2.8   
9655             Ed Davis  26.0  81.0   0.0  20.8  0.611  0.0  NaN  4.2   
9670             Ed Davis  27.0  46.0  12.0  17.2  0.528  0.0  NaN  3.1   

        2P%   eFG%  FTA    FT%  ORB  TRB  AST  STL  BLK  TOV   PF  PTS team  \
13    0.600  0.600  

Like our FT%, the missing 3P% is dued to the fact that these players didn't attempt any 3-pointers at all in the seasons, resulting 3P% null values. Therefore I will do the same method and set them as 0s.

In [99]:
combined_df['3P%'] = combined_df['3P%'].fillna(0)
#check
combined_df.isna().sum()

Player    0
Age       0
G         0
GS        0
MP        0
FG%       0
3PA       0
3P%       0
2PA       0
2P%       0
eFG%      0
FTA       0
FT%       0
ORB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS       0
team      0
season    0
Pos       0
Ht        0
Wt        0
Exp       0
PER       0
TS%       0
3PAr      0
AST%      0
STL%      0
BLK%      0
TOV%      0
USG%      0
OWS       0
WS        0
WS/48     0
BPM       0
OBPM      0
VORP      0
TRB%      0
dtype: int64

In [100]:
combined_df.shape

(13521, 42)

After data cleaning, our dataframe now has 9771 rows and 45 columns

# Data Preprocessing and Feature Enginneering

In [101]:
# With height, they are stored as objects in the dataframes, in feets and inches. I need them as floats to do EDA on them. Therefore I want to convert them to cms
def height_to_cm(height):
    feet, inches = map(int, height.split('-'))
    return (feet * 30.48) + (inches * 2.54)

combined_df['Ht_cm'] = combined_df['Ht'].apply(height_to_cm)

combined_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm
0,Isaiah Rider,23.0,75.0,67.0,35.3,0.447,5.3,0.351,11.4,0.491,0.502,4.5,0.817,1.2,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8,195.58
1,Christian Laettner,25.0,81.0,80.0,34.2,0.489,0.5,0.325,10.9,0.497,0.496,6.2,0.818,2.0,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6,210.82
2,Doug West,27.0,71.0,65.0,32.8,0.461,0.9,0.180,9.9,0.485,0.468,3.5,0.837,0.8,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0,198.12
3,Tom Gugliotta,25.0,31.0,17.0,32.8,0.454,2.8,0.318,8.7,0.498,0.493,3.9,0.762,1.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4,208.28
4,Sean Rooks,25.0,80.0,70.0,30.1,0.470,0.1,0.000,7.6,0.474,0.470,4.8,0.761,2.1,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4,208.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13516,Kevin Durant,30.0,78.0,78.0,34.6,0.521,5.0,0.353,12.8,0.587,0.571,6.5,0.885,0.4,6.4,5.9,0.7,1.1,2.9,2.0,26.0,GSW,2019/2020,SF,6-11,240,11,24.2,0.631,0.281,26.2,1.0,2.6,12.3,29.0,8.6,11.5,0.204,5.5,5.4,5.1,9.8,210.82
13517,Kevin Durant,32.0,35.0,32.0,33.1,0.537,5.4,0.450,11.8,0.577,0.608,6.8,0.882,0.4,7.1,5.6,0.7,1.3,3.4,2.0,26.9,BRK,2021/2022,PF,6-11,240,12,26.4,0.666,0.313,27.5,1.0,3.4,14.5,31.2,3.7,5.0,0.206,7.2,6.4,2.7,11.8,210.82
13518,Kevin Durant,33.0,55.0,55.0,37.2,0.518,5.5,0.383,14.8,0.568,0.570,7.4,0.910,0.5,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8,210.82
13519,Kevin Durant,34.0,39.0,39.0,36.0,0.559,4.8,0.376,14.0,0.622,0.607,7.3,0.934,0.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5,210.82


In [102]:
# Since some positions are very similar in heights, weights and even playstyles, I want to consolidate them to 1 position. These will be G (guards) which will include PG and SG, F (forwards) which will include SF and PF, and we will keep C.
# Now we will have 3 different values for our "Pos" column. Then I want to convert the "Pos" column to dummy columns so I can run linear regression on the model later.


positions_map = {
    'PG': 'G',  # Point Guard to Guard
    'SG': 'G',  # Shooting Guard to Guard
    'SF': 'F',  # Small Forward to Forward
    'PF': 'F',  # Power Forward to Forward
    'C': 'C'    # Center remains Center
}

combined_df['grouped_Pos'] = combined_df['Pos'].map(positions_map) # Apply the mapping to the 'Pos' column

print(combined_df[['Pos', 'grouped_Pos']].head(10)) # Check the transformation

  Pos grouped_Pos
0  SG           G
1   C           C
2  SG           G
3  SF           F
4   C           C
5  PG           G
6  SF           F
7  PG           G
8  SG           G
9  PF           F


In [103]:
pos_dummy = pd.get_dummies(combined_df["grouped_Pos"], prefix = "Pos type dummy") #Made the position column into dummy variables

numerical_df = pd.concat([combined_df, pos_dummy], axis=1) #Concating the new position dummy columns into the table and make a new dataframe to keep the original datafrane
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G
0,Isaiah Rider,23.0,75.0,67.0,35.3,0.447,5.3,0.351,11.4,0.491,0.502,4.5,0.817,1.2,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8,195.58,G,0,0,1
1,Christian Laettner,25.0,81.0,80.0,34.2,0.489,0.5,0.325,10.9,0.497,0.496,6.2,0.818,2.0,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6,210.82,C,1,0,0
2,Doug West,27.0,71.0,65.0,32.8,0.461,0.9,0.180,9.9,0.485,0.468,3.5,0.837,0.8,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0,198.12,G,0,0,1
3,Tom Gugliotta,25.0,31.0,17.0,32.8,0.454,2.8,0.318,8.7,0.498,0.493,3.9,0.762,1.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4,208.28,F,0,1,0
4,Sean Rooks,25.0,80.0,70.0,30.1,0.470,0.1,0.000,7.6,0.474,0.470,4.8,0.761,2.1,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4,208.28,C,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13516,Kevin Durant,30.0,78.0,78.0,34.6,0.521,5.0,0.353,12.8,0.587,0.571,6.5,0.885,0.4,6.4,5.9,0.7,1.1,2.9,2.0,26.0,GSW,2019/2020,SF,6-11,240,11,24.2,0.631,0.281,26.2,1.0,2.6,12.3,29.0,8.6,11.5,0.204,5.5,5.4,5.1,9.8,210.82,F,0,1,0
13517,Kevin Durant,32.0,35.0,32.0,33.1,0.537,5.4,0.450,11.8,0.577,0.608,6.8,0.882,0.4,7.1,5.6,0.7,1.3,3.4,2.0,26.9,BRK,2021/2022,PF,6-11,240,12,26.4,0.666,0.313,27.5,1.0,3.4,14.5,31.2,3.7,5.0,0.206,7.2,6.4,2.7,11.8,210.82,F,0,1,0
13518,Kevin Durant,33.0,55.0,55.0,37.2,0.518,5.5,0.383,14.8,0.568,0.570,7.4,0.910,0.5,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8,210.82,F,0,1,0
13519,Kevin Durant,34.0,39.0,39.0,36.0,0.559,4.8,0.376,14.0,0.622,0.607,7.3,0.934,0.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5,210.82,F,0,1,0


More feature engineerings

Since I am trying to predict the improvement of players, I believe their past performances are important metrics too. Our dataframe consist rows of separate individual seasons of players, so I want to use the groupby function to make some important "previous" season stats as new columns.

I am going to first add an extra feature "Past PER average", which is the average PER of a player from all their previous seasons played

In [104]:
numerical_df['cumulative_sum_PER'] = numerical_df.groupby('Player')['PER'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous PER average'] = (numerical_df['cumulative_sum_PER'] - numerical_df['PER']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous PER average'] = numerical_df['Previous PER average'].fillna(numerical_df["PER"]) # Fill in the null values which is caused by players with no previous seasons. I am going to fill in with their current season PER as their average,
numerical_df.drop(['cumulative_sum_PER', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

Adding an extra feature "Previous PER improvement", which is the difference of the current season's and previous year's PER, which is essentially a player's last year's PER Improvement

In [105]:
numerical_df['Previous PER Improvement'] = numerical_df.groupby('Player')['PER'].diff()
numerical_df['Previous PER Improvement'].fillna(0, inplace=True) # We can fillin NA with 0s because the average PER Improvement of the league is 0 too.

Adding an extra column "Past PER improvement average", which is the average PER Improvement of a player from all their previous seasons played.

In [106]:
numerical_df['cumulative_PER_Improvement'] = numerical_df.groupby('Player')['Previous PER Improvement'].cumsum()
numerical_df['seasons_count'] = numerical_df.groupby('Player').cumcount()
numerical_df['Previous PER Improvement Average'] = numerical_df['cumulative_PER_Improvement'] / numerical_df['seasons_count']
numerical_df.drop(['cumulative_PER_Improvement', 'seasons_count'], axis=1, inplace=True)
numerical_df['Previous PER Improvement Average'].fillna(0, inplace=True)

I have realised these past stats of a player are quite significant for predicting a players' improvement, so I decided to create more of them with features I find important from my modelings

In [107]:
numerical_df['Previous OBPM Improvement'] = numerical_df.groupby('Player')['OBPM'].diff()
numerical_df['Previous OBPM Improvement'].fillna(0, inplace=True)

Creating more features which are 'Previous OBPM average', 'Previous OBPM Improvement Average', 'Previous TS% average', 'Previous PTS average', 'Previous USG% Improvement', 'Previous USG% average', 'Previous 3PAr average' and 'Previous FG% average'. They all look into the historical stats of players so I can improve the complexity of my model.

In [108]:
numerical_df['cumulative_sum_OBPM'] = numerical_df.groupby('Player')['OBPM'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous OBPM average'] = (numerical_df['cumulative_sum_OBPM'] - numerical_df['OBPM']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous OBPM average'] = numerical_df['Previous OBPM average'].fillna(numerical_df["OBPM"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_OBPM', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [109]:
numerical_df['cumulative_OBPM_Improvement'] = numerical_df.groupby('Player')['Previous OBPM Improvement'].cumsum()
numerical_df['seasons_count'] = numerical_df.groupby('Player').cumcount()
numerical_df['Previous OBPM Improvement Average'] = numerical_df['cumulative_OBPM_Improvement'] / numerical_df['seasons_count']
numerical_df.drop(['cumulative_OBPM_Improvement', 'seasons_count'], axis=1, inplace=True)
numerical_df['Previous OBPM Improvement Average'].fillna(0, inplace=True)

In [110]:
numerical_df['cumulative_sum_TS%'] = numerical_df.groupby('Player')['TS%'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous TS% average'] = (numerical_df['cumulative_sum_TS%'] - numerical_df['TS%']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous TS% average'] = numerical_df['Previous TS% average'].fillna(numerical_df["TS%"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_TS%', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [111]:
numerical_df['cumulative_sum_PTS'] = numerical_df.groupby('Player')['PTS'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous PTS average'] = (numerical_df['cumulative_sum_PTS'] - numerical_df['PTS']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous PTS average'] = numerical_df['Previous PTS average'].fillna(numerical_df["PTS"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_PTS', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [112]:
numerical_df['Previous USG% Improvement'] = numerical_df.groupby('Player')['USG%'].diff()
numerical_df['Previous USG% Improvement'].fillna(0, inplace=True)

In [113]:
numerical_df['cumulative_sum_USG%'] = numerical_df.groupby('Player')['USG%'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous USG% average'] = (numerical_df['cumulative_sum_USG%'] - numerical_df['USG%']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous USG% average'] = numerical_df['Previous USG% average'].fillna(numerical_df["USG%"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_USG%', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [114]:
numerical_df['cumulative_sum_3PAr'] = numerical_df.groupby('Player')['3PAr'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous 3PAr average'] = (numerical_df['cumulative_sum_3PAr'] - numerical_df['3PAr']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous 3PAr average'] = numerical_df['Previous 3PAr average'].fillna(numerical_df["3PAr"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_3PAr', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [115]:
numerical_df['cumulative_sum_FG%'] = numerical_df.groupby('Player')['FG%'].cumsum() # Using cumsum to calcualte the cumulative sum
numerical_df['cumulative_count'] = numerical_df.groupby('Player').cumcount() + 1 # Plus 1 because cumcount start from 0
numerical_df['Previous FG% average'] = (numerical_df['cumulative_sum_FG%'] - numerical_df['FG%']) / (numerical_df['cumulative_count'] - 1) # -1 because - the current season
numerical_df['Previous FG% average'] = numerical_df['Previous FG% average'].fillna(numerical_df["FG%"]) # Fill in the null values because of the s
numerical_df.drop(['cumulative_sum_FG%', 'cumulative_count'], axis=1, inplace=True) # Dropping the calculation columns

In [116]:
numerical_df = numerical_df.reset_index(drop=True)
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average
0,Isaiah Rider,23.0,75.0,67.0,35.3,0.447,5.3,0.351,11.4,0.491,0.502,4.5,0.817,1.2,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215,1,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8,195.58,G,0,0,1,15.600000,0.0,0.000000,0.0,2.000000,0.000000,0.548000,20.400000,0.0,28.200000,0.317000,0.447000
1,Christian Laettner,25.0,81.0,80.0,34.2,0.489,0.5,0.325,10.9,0.497,0.496,6.2,0.818,2.0,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235,2,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6,210.82,C,1,0,0,17.700000,0.0,0.000000,0.0,0.700000,0.000000,0.580000,16.300000,0.0,22.500000,0.043000,0.489000
2,Doug West,27.0,71.0,65.0,32.8,0.461,0.9,0.180,9.9,0.485,0.468,3.5,0.837,0.8,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200,5,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0,198.12,G,0,0,1,11.600000,0.0,0.000000,0.0,-2.200000,0.000000,0.528000,12.900000,0.0,19.600000,0.080000,0.461000
3,Tom Gugliotta,25.0,31.0,17.0,32.8,0.454,2.8,0.318,8.7,0.498,0.493,3.9,0.762,1.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240,2,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4,208.28,F,0,1,0,18.400000,0.0,0.000000,0.0,2.200000,0.000000,0.542000,14.400000,0.0,22.100000,0.246000,0.454000
4,Sean Rooks,25.0,80.0,70.0,30.1,0.470,0.1,0.000,7.6,0.474,0.470,4.8,0.761,2.1,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250,2,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4,208.28,C,1,0,0,12.600000,0.0,0.000000,0.0,-1.500000,0.000000,0.555000,10.900000,0.0,17.600000,0.008000,0.470000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13516,Kevin Durant,30.0,78.0,78.0,34.6,0.521,5.0,0.353,12.8,0.587,0.571,6.5,0.885,0.4,6.4,5.9,0.7,1.1,2.9,2.0,26.0,GSW,2019/2020,SF,6-11,240,11,24.2,0.631,0.281,26.2,1.0,2.6,12.3,29.0,8.6,11.5,0.204,5.5,5.4,5.1,9.8,210.82,F,0,1,0,25.463636,-1.8,0.763636,-1.3,6.218182,0.445455,0.612909,26.963636,-1.4,30.090909,0.264818,0.492818
13517,Kevin Durant,32.0,35.0,32.0,33.1,0.537,5.4,0.450,11.8,0.577,0.608,6.8,0.882,0.4,7.1,5.6,0.7,1.3,3.4,2.0,26.9,BRK,2021/2022,PF,6-11,240,12,26.4,0.666,0.313,27.5,1.0,3.4,14.5,31.2,3.7,5.0,0.206,7.2,6.4,2.7,11.8,210.82,F,0,1,0,25.358333,2.2,0.883333,1.0,6.150000,0.491667,0.614417,26.883333,2.2,30.000000,0.266167,0.495167
13518,Kevin Durant,33.0,55.0,55.0,37.2,0.518,5.5,0.383,14.8,0.568,0.570,7.4,0.910,0.5,7.4,6.4,0.9,0.9,3.5,2.1,29.9,BRK,2022/2023,PF,6-11,240,13,25.6,0.634,0.269,29.1,1.1,2.3,12.9,31.2,6.4,8.4,0.198,7.2,6.4,4.8,10.8,210.82,F,0,1,0,25.438462,-0.8,0.753846,0.0,6.169231,0.453846,0.618385,26.884615,0.0,30.092308,0.269769,0.498385
13519,Kevin Durant,34.0,39.0,39.0,36.0,0.559,4.8,0.376,14.0,0.622,0.607,7.3,0.934,0.4,6.7,5.3,0.8,1.5,3.5,2.4,29.7,BRK,2023/2024,PF,6-11,240,14,26.2,0.673,0.257,25.9,1.1,3.5,13.7,31.5,3.9,5.7,0.194,7.0,5.8,3.2,10.5,210.82,F,0,1,0,25.450000,0.6,0.742857,-0.6,6.185714,0.378571,0.619500,27.100000,0.3,30.171429,0.269714,0.499786


Here I want to webscrap the current season of NBA, which just finished.

In [117]:
tables = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2024_advanced.html')


df = tables[0]
df = df.drop(columns=['Rk'])
df = df.reset_index(drop=True)
df

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,Precious Achiuwa,PF-C,24,TOT,74,1624,14.6,.545,.207,.239,13.0,20.5,16.7,8.4,1.4,3.9,13.8,15.9,NaN,1.2,2.2,3.4,.102,NaN,-1.7,0.3,-1.4,0.2
1,Precious Achiuwa,C,24,TOR,25,437,15.0,.512,.276,.247,12.3,22.1,17.1,14.5,1.8,2.4,13.3,21.2,NaN,0.0,0.4,0.4,.048,NaN,-1.4,-0.2,-1.6,0.0
2,Precious Achiuwa,PF,24,NYK,49,1187,14.5,.564,.167,.234,13.3,19.9,16.6,6.2,1.3,4.4,14.1,14.0,NaN,1.2,1.8,3.0,.122,NaN,-1.9,0.5,-1.4,0.2
3,Bam Adebayo,C,26,MIA,71,2416,19.8,.576,.041,.381,7.4,27.3,17.3,18.9,1.7,2.7,12.0,24.9,NaN,2.9,4.3,7.2,.144,NaN,0.8,1.7,2.4,2.7
4,Ochai Agbaji,SG,23,TOT,78,1641,7.7,.497,.487,.129,4.9,9.6,7.2,6.6,1.4,2.4,12.3,13.4,NaN,-0.5,0.6,0.1,.002,NaN,-3.5,-0.9,-4.4,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758,Thaddeus Young,PF,35,PHO,10,89,15.1,.515,.048,.143,22.8,13.3,17.8,10.4,2.7,2.0,15.2,12.8,NaN,0.1,0.1,0.3,.137,NaN,-1.5,0.6,-0.9,0.0
759,Trae Young,PG,25,ATL,54,1942,20.3,.585,.465,.404,1.2,7.3,4.2,45.1,1.8,0.5,16.5,30.5,NaN,4.0,0.6,4.6,.114,NaN,4.9,-2.3,2.6,2.2
760,Omer Yurtseven,C,25,UTA,48,545,15.0,.565,.130,.152,14.7,27.3,21.0,7.7,0.7,2.9,15.9,17.9,NaN,0.3,0.4,0.7,.062,NaN,-1.6,-1.5,-3.0,-0.1
761,Cody Zeller,C,31,NOP,43,320,12.8,.483,.048,.613,16.9,22.0,19.5,15.2,1.4,1.5,16.9,12.9,NaN,0.4,0.4,0.8,.124,NaN,-2.9,0.3,-2.6,0.0


In [118]:
df.duplicated().sum()

27

In [119]:
duplicate_rows = df.duplicated(keep='first')
print("Duplicate Rows (excluding the first occurrence):")
print(df[duplicate_rows])

Duplicate Rows (excluding the first occurrence):
     Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%  \
59   Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
92   Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
119  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
140  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
165  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
190  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
222  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
256  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
279  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
304  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB%  DRB%  TRB%  AST%   
327  Player  Pos  Age  Tm  G  MP  PER  TS%  3PAr  FTr  ORB% 

In [120]:
#We are going to drop every row except the first row which is our header

rows_to_drop = [59, 90, 117, 138, 163, 188, 220, 252, 275, 300, 323, 346, 367, 391, 423, 452, 482, 512, 541, 564, 591, 616, 641, 666, 689, 716, 742]
df = df.drop(rows_to_drop)
df.reset_index(drop=True, inplace=True)
df['season'] = '2024/2025'
df.rename(columns={'Tm': 'team'}, inplace=True)
df

,Player,Pos,Age,team,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,season
0,Precious Achiuwa,PF-C,24,TOT,74,1624,14.6,.545,.207,.239,13.0,20.5,16.7,8.4,1.4,3.9,13.8,15.9,NaN,1.2,2.2,3.4,.102,NaN,-1.7,0.3,-1.4,0.2,2024/2025
1,Precious Achiuwa,C,24,TOR,25,437,15.0,.512,.276,.247,12.3,22.1,17.1,14.5,1.8,2.4,13.3,21.2,NaN,0.0,0.4,0.4,.048,NaN,-1.4,-0.2,-1.6,0.0,2024/2025
2,Precious Achiuwa,PF,24,NYK,49,1187,14.5,.564,.167,.234,13.3,19.9,16.6,6.2,1.3,4.4,14.1,14.0,NaN,1.2,1.8,3.0,.122,NaN,-1.9,0.5,-1.4,0.2,2024/2025
3,Bam Adebayo,C,26,MIA,71,2416,19.8,.576,.041,.381,7.4,27.3,17.3,18.9,1.7,2.7,12.0,24.9,NaN,2.9,4.3,7.2,.144,NaN,0.8,1.7,2.4,2.7,2024/2025
4,Ochai Agbaji,SG,23,TOT,78,1641,7.7,.497,.487,.129,4.9,9.6,7.2,6.6,1.4,2.4,12.3,13.4,NaN,-0.5,0.6,0.1,.002,NaN,-3.5,-0.9,-4.4,-1.0,2024/2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,Thaddeus Young,PF,35,PHO,10,89,15.1,.515,.048,.143,22.8,13.3,17.8,10.4,2.7,2.0,15.2,12.8,NaN,0.1,0.1,0.3,.137,NaN,-1.5,0.6,-0.9,0.0,2024/2025
732,Trae Young,PG,25,ATL,54,1942,20.3,.585,.465,.404,1.2,7.3,4.2,45.1,1.8,0.5,16.5,30.5,NaN,4.0,0.6,4.6,.114,NaN,4.9,-2.3,2.6,2.2,2024/2025
733,Omer Yurtseven,C,25,UTA,48,545,15.0,.565,.130,.152,14.7,27.3,21.0,7.7,0.7,2.9,15.9,17.9,NaN,0.3,0.4,0.7,.062,NaN,-1.6,-1.5,-3.0,-0.1,2024/2025
734,Cody Zeller,C,31,NOP,43,320,12.8,.483,.048,.613,16.9,22.0,19.5,15.2,1.4,1.5,16.9,12.9,NaN,0.4,0.4,0.8,.124,NaN,-2.9,0.3,-2.6,0.0,2024/2025


In [121]:
df.duplicated().sum()

25

In [122]:
df_selected = df[["Player", "team", "season", "PER"]]
df_selected

,Player,team,season,PER
0,Precious Achiuwa,TOT,2024/2025,14.6
1,Precious Achiuwa,TOR,2024/2025,15.0
2,Precious Achiuwa,NYK,2024/2025,14.5
3,Bam Adebayo,MIA,2024/2025,19.8
4,Ochai Agbaji,TOT,2024/2025,7.7
...,...,...,...,...
731,Thaddeus Young,PHO,2024/2025,15.1
732,Trae Young,ATL,2024/2025,20.3
733,Omer Yurtseven,UTA,2024/2025,15.0
734,Cody Zeller,NOP,2024/2025,12.8


In [123]:
df_selected['PER'] = pd.to_numeric(df_selected['PER'], errors='coerce')
df_selected

C:\Users\ivant\AppData\Local\Temp\ipykernel_3152\719399426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['PER'] = pd.to_numeric(df_selected['PER'], errors='coerce')


,Player,team,season,PER
0,Precious Achiuwa,TOT,2024/2025,14.6
1,Precious Achiuwa,TOR,2024/2025,15.0
2,Precious Achiuwa,NYK,2024/2025,14.5
3,Bam Adebayo,MIA,2024/2025,19.8
4,Ochai Agbaji,TOT,2024/2025,7.7
...,...,...,...,...
731,Thaddeus Young,PHO,2024/2025,15.1
732,Trae Young,ATL,2024/2025,20.3
733,Omer Yurtseven,UTA,2024/2025,15.0
734,Cody Zeller,NOP,2024/2025,12.8


In [124]:
missing = df_selected[df_selected['PER'].isna()]
df_selected = df_selected.dropna()
df_selected

,Player,team,season,PER
0,Precious Achiuwa,TOT,2024/2025,14.6
1,Precious Achiuwa,TOR,2024/2025,15.0
2,Precious Achiuwa,NYK,2024/2025,14.5
3,Bam Adebayo,MIA,2024/2025,19.8
4,Ochai Agbaji,TOT,2024/2025,7.7
...,...,...,...,...
731,Thaddeus Young,PHO,2024/2025,15.1
732,Trae Young,ATL,2024/2025,20.3
733,Omer Yurtseven,UTA,2024/2025,15.0
734,Cody Zeller,NOP,2024/2025,12.8


In [125]:
numerical_df = pd.merge(numerical_df, df_selected, on=["Player", "team", "season", "PER"], how="outer", suffixes=('_numerical', '_selected'))
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average
0,Isaiah Rider,23.0,75.0,67.0,35.3,0.447,5.3,0.351,11.4,0.491,0.502,4.5,0.817,1.2,3.3,3.3,0.9,0.3,3.1,2.6,20.4,MIN,1995/1996,SG,6-5,215.0,1.0,15.6,0.548,0.317,18.7,1.4,0.7,14.2,28.2,1.4,1.9,0.034,-0.5,2.0,1.0,5.8,195.58,G,0.0,0.0,1.0,15.6,0.0,0.0,0.0,2.0,0.0,0.548,20.4,0.0,28.2,0.317,0.447
1,Christian Laettner,25.0,81.0,80.0,34.2,0.489,0.5,0.325,10.9,0.497,0.496,6.2,0.818,2.0,7.6,2.9,1.2,1.1,2.8,3.7,16.3,MIN,1995/1996,C,6-11,235.0,2.0,17.7,0.580,0.043,15.5,1.9,2.4,16.5,22.5,3.6,6.2,0.108,1.1,0.7,2.2,13.6,210.82,C,1.0,0.0,0.0,17.7,0.0,0.0,0.0,0.7,0.0,0.580,16.3,0.0,22.5,0.043,0.489
2,Doug West,27.0,71.0,65.0,32.8,0.461,0.9,0.180,9.9,0.485,0.468,3.5,0.837,0.8,3.2,2.6,0.9,0.3,1.8,3.5,12.9,MIN,1995/1996,SG,6-6,200.0,5.0,11.6,0.528,0.080,14.3,1.5,0.8,12.6,19.6,1.2,1.8,0.038,-3.4,-2.2,-0.8,6.0,198.12,G,0.0,0.0,1.0,11.6,0.0,0.0,0.0,-2.2,0.0,0.528,12.9,0.0,19.6,0.080,0.461
3,Tom Gugliotta,25.0,31.0,17.0,32.8,0.454,2.8,0.318,8.7,0.498,0.493,3.9,0.762,1.6,7.2,4.5,2.0,0.9,2.6,2.8,14.4,MIN,1995/1996,SF,6-10,240.0,2.0,18.4,0.542,0.246,25.0,3.1,2.1,16.5,22.1,0.8,2.1,0.097,3.6,2.2,1.4,13.4,208.28,F,0.0,1.0,0.0,18.4,0.0,0.0,0.0,2.2,0.0,0.542,14.4,0.0,22.1,0.246,0.454
4,Sean Rooks,25.0,80.0,70.0,30.1,0.470,0.1,0.000,7.6,0.474,0.470,4.8,0.761,2.1,6.1,1.2,0.4,0.9,1.8,2.6,10.9,MIN,1995/1996,C,6-10,250.0,2.0,12.6,0.555,0.008,6.9,0.6,2.2,15.4,17.6,1.9,3.0,0.060,-3.0,-1.5,-0.6,12.4,208.28,C,1.0,0.0,0.0,12.6,0.0,0.0,0.0,-1.5,0.0,0.555,10.9,0.0,17.6,0.008,0.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14224,Thaddeus Young,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHO,2024/2025,NaN,NaN,NaN,NaN,15.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14225,Trae Young,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATL,2024/2025,NaN,NaN,NaN,NaN,20.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14226,Omer Yurtseven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UTA,2024/2025,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14227,Cody Zeller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NOP,2024/2025,NaN,NaN,NaN,NaN,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Choosing our target variable

To measure a player's performance then their improvement, we need a good metric that can include everything a player do on court, not just one-dimensonal metrics like "Points" or "Rebound". Therefore we want to look into the advanced NBA metrics like "PER", "BPM", "WS", "WS/48", they take into most stats of a players' contribution and usually compare to league averages for fair comparison.

The Player Efficiency Rating (PER) was a creation of ESPN basketball columnist John Hollinger. In simpler terms, PER scores include basic statistics like field goals made, steals, three-pointers made, and then subtract things like free throws missed, turnovers, and fouls. Then compare with team stats and league averages. Hollinger sets the league average PER to 15.00 every season to standardize the metric.

BPM uses a player’s box score information, position, and the team’s overall performance to estimate the player’s contribution in points above league average per 100 possessions played. BPM does not take into account playing time.

To calculate the Win Shares, it requires dividing the sum of the player’s marginal point production by his team’s marginal points per win. In this way, a team’s Win Shares is distributed in a way that sums to the team’s actual win totals. There is one fundamental flaw with win shares – they rely on ‘counting stats’ that aren’t weighted by playing time. As such, they heavily favor players who spend more time on the court. This is where win shares per 48 minutes (WS/48) comes in. "WS/48" levels the playing field to judge the impact of each player if they all played the same number of minutes. WS/48 = (Winshares/Minutes played)* 48

As players and teams are all playing to win, for the first seed and ultimately the championship, a player's WS/48 is a very good measurement in terms of how much they are helping their team to win. However ws/48 also relies hugely on how the whole team is performing, if a team is winning a lot their key player will typically have high WS/48 too. We might come back to this measurement in the future

Therefore, I decided to look at PER, PER is a very strong measurement to determine how well the player is impacting the court, and its much less impacted by the team but more about the individual. It also compare to the league averages and team stats.

I want to use the improvement of PER of a player as our target variable, so I want to make a new target column "PER Improvement" which calculates the difference of a player's current year's and next year's PERs

In [126]:
numerical_df.sort_values(by=['Player', 'season'], inplace=True) # I have to sort the table so that rows for the same players are put together, with season in ascending order, 
#then we can find the differences of PER of consecutive years as our PER improvement stats.
numerical_df['PER Improvement'] = numerical_df.groupby('Player')['PER'].diff(-1)*-1 # I am doing diff(-1) to find the difference between next year's and this year's PER 
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average,PER Improvement
12218,A.C. Green,23.0,79.0,72.0,28.4,0.538,0.1,0.000,7.4,0.543,0.538,3.6,0.780,2.7,7.8,1.1,0.9,1.0,1.3,2.2,10.8,LAL,1987/1988,PF,6-9,220.0,1.0,15.7,0.599,0.009,4.6,1.5,2.0,12.5,14.7,4.3,7.6,0.163,1.0,0.9,1.7,15.3,205.74,F,0.0,1.0,0.0,13.986667,1.2,0.300000,0.5,0.006667,0.206667,0.539867,9.453333,0.0,14.540000,0.054467,0.481200,-1.2
11925,A.C. Green,24.0,82.0,64.0,32.1,0.503,0.0,0.000,7.8,0.505,0.503,4.6,0.773,3.0,8.7,1.1,1.1,0.5,1.5,2.5,11.4,LAL,1988/1989,PF,6-9,220.0,2.0,14.5,0.581,0.003,4.5,1.6,1.0,12.9,14.7,4.5,7.9,0.144,0.3,0.4,1.5,15.3,205.74,F,0.0,1.0,0.0,13.950000,-3.3,0.235714,-1.4,-0.021429,0.185714,0.536929,9.314286,-2.3,14.528571,0.058143,0.479643,3.3
11600,A.C. Green,25.0,82.0,82.0,30.6,0.529,0.2,0.235,9.0,0.536,0.532,4.4,0.786,3.1,9.0,1.3,1.1,0.7,1.5,2.1,13.3,LAL,1989/1990,PF,6-9,220.0,3.0,17.8,0.594,0.022,5.5,1.8,1.2,11.5,17.0,5.8,9.4,0.179,1.6,1.8,2.3,16.4,205.74,F,0.0,1.0,0.0,13.653846,3.1,0.507692,1.5,-0.161538,0.307692,0.532538,9.007692,-0.1,14.338462,0.060923,0.475846,-3.1
11264,A.C. Green,26.0,82.0,82.0,33.0,0.478,0.6,0.283,9.3,0.489,0.486,4.5,0.751,3.2,8.7,1.1,0.8,0.6,1.4,2.5,12.9,LAL,1990/1991,PF,6-9,220.0,4.0,14.7,0.548,0.057,4.6,1.2,1.1,10.7,17.1,4.4,7.7,0.137,-0.6,0.3,1.0,15.1,205.74,F,0.0,1.0,0.0,13.566667,0.9,0.291667,0.2,-0.200000,0.208333,0.531250,8.683333,1.5,14.108333,0.061250,0.475667,-0.9
10926,A.C. Green,27.0,82.0,21.0,26.4,0.476,0.7,0.200,5.9,0.507,0.486,3.7,0.738,2.5,6.3,0.9,0.7,0.3,1.2,1.4,9.1,LAL,1991/1992,PF,6-9,220.0,5.0,13.8,0.556,0.101,4.5,1.4,0.6,12.8,15.6,3.2,6.1,0.135,0.0,0.1,1.1,13.8,205.74,F,0.0,1.0,0.0,13.545455,-2.9,0.236364,-1.6,-0.227273,0.209091,0.529000,8.645455,-1.2,13.972727,0.057636,0.475636,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8282,Željko Rebrača,29.0,74.0,4.0,15.9,0.505,0.0,0.000,5.1,0.505,0.505,2.4,0.771,1.1,3.9,0.5,0.4,1.0,1.1,2.6,6.9,DET,2002/2003,C,7-0,257.0,0.0,16.2,0.569,0.000,5.7,1.3,4.4,15.7,21.4,1.4,3.3,0.134,-2.3,-2.9,-0.1,14.5,213.36,C,1.0,0.0,0.0,12.350000,5.5,1.100000,0.8,-2.700000,-0.600000,0.608500,5.250000,5.9,15.750000,0.000000,0.555000,-2.2
8293,Željko Rebrača,30.0,30.0,12.0,16.3,0.552,0.0,0.000,4.8,0.552,0.552,1.6,0.792,0.9,3.1,0.3,0.2,0.6,1.0,2.6,6.6,DET,2003/2004,C,7-0,257.0,1.0,14.0,0.596,0.000,3.6,0.7,2.6,14.9,19.2,0.7,1.3,0.133,-2.6,-2.8,-0.1,11.2,213.36,C,1.0,0.0,0.0,13.633333,-2.2,0.000000,0.1,-2.766667,-0.366667,0.595333,5.800000,-2.2,17.633333,0.000000,0.538333,-4.4
8308,Željko Rebrača,31.0,21.0,2.0,10.6,0.407,0.0,0.000,2.6,0.407,0.407,1.3,0.786,0.9,2.3,0.2,0.2,0.4,0.6,2.1,3.1,DET,2004/2005,C,7-0,257.0,2.0,9.6,0.498,0.000,3.0,1.2,3.1,16.4,16.7,0.1,0.5,0.115,-3.9,-5.1,-0.1,12.8,213.36,C,1.0,0.0,0.0,13.725000,-4.4,-1.100000,-2.3,-2.775000,-0.850000,0.595500,6.000000,-2.5,18.025000,0.000000,0.541750,4.4
3255,Željko Rebrača,32.0,58.0,2.0,16.0,0.568,0.0,0.000,4.0,0.568,0.568,1.5,0.859,0.8,3.2,0.4,0.2,0.7,0.8,2.2,5.8,LAC,2005/2006,C,7-0,257.0,3.0,14.0,0.625,0.000,4.8,0.8,3.3,15.3,16.0,1.4,2.4,0.122,-1.3,-1.7,0.2,11.9,213.36,C,1.0,0.0,0.0,14.000000,0.0,0.000000,0.0,-1.700000,0.000000,0.625000,5.800000,0.0,16.000000,0.000000,0.568000,-3.3


In [127]:
numerical_df = numerical_df[numerical_df['season'] != '2024/2025']

In [128]:
numerical_df.isna().sum() #We should have more null data in our dataset now as PER Improvement will be null for players in their final year.

Player                                  0
Age                                     0
G                                       0
GS                                      0
MP                                      0
FG%                                     0
3PA                                     0
3P%                                     0
2PA                                     0
2P%                                     0
eFG%                                    0
FTA                                     0
FT%                                     0
ORB                                     0
TRB                                     0
AST                                     0
STL                                     0
BLK                                     0
TOV                                     0
PF                                      0
PTS                                     0
team                                    0
season                                  0
Pos                               

Going back to the null values for our target column "PER Improvement", my original idea was to set them to 0 as well. However considering it is our target feature, and setting nearly 2000 of our target values to 0 when we only have 9771 rows, this will greatly affect our results for modelling later, leading to greater inaccuracy. Therefore I decided to drop the rows with N/A PER Improvement, which are basically the final years of a players career, of the latest season where they don't have a next year to calculate PER Improvement. This will reduce our dataset by a lot, but it is still better than hugely affecting our target data for modelling.

In [129]:
numerical_df = numerical_df.dropna(subset=['PER Improvement']) # Dropping the rows with NA PER Improvement

In [130]:
numerical_df.isna().sum()

Player                               0
Age                                  0
G                                    0
GS                                   0
MP                                   0
FG%                                  0
3PA                                  0
3P%                                  0
2PA                                  0
2P%                                  0
eFG%                                 0
FTA                                  0
FT%                                  0
ORB                                  0
TRB                                  0
AST                                  0
STL                                  0
BLK                                  0
TOV                                  0
PF                                   0
PTS                                  0
team                                 0
season                               0
Pos                                  0
Ht                                   0
Wt                       

In [131]:
numerical_df

,Player,Age,G,GS,MP,FG%,3PA,3P%,2PA,2P%,eFG%,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,team,season,Pos,Ht,Wt,Exp,PER,TS%,3PAr,AST%,STL%,BLK%,TOV%,USG%,OWS,WS,WS/48,BPM,OBPM,VORP,TRB%,Ht_cm,grouped_Pos,Pos type dummy_C,Pos type dummy_F,Pos type dummy_G,Previous PER average,Previous PER Improvement,Previous PER Improvement Average,Previous OBPM Improvement,Previous OBPM average,Previous OBPM Improvement Average,Previous TS% average,Previous PTS average,Previous USG% Improvement,Previous USG% average,Previous 3PAr average,Previous FG% average,PER Improvement
12218,A.C. Green,23.0,79.0,72.0,28.4,0.538,0.1,0.000,7.4,0.543,0.538,3.6,0.780,2.7,7.8,1.1,0.9,1.0,1.3,2.2,10.8,LAL,1987/1988,PF,6-9,220.0,1.0,15.7,0.599,0.009,4.6,1.5,2.0,12.5,14.7,4.3,7.6,0.163,1.0,0.9,1.7,15.3,205.74,F,0.0,1.0,0.0,13.986667,1.2,0.300000,0.5,0.006667,0.206667,0.539867,9.453333,0.0,14.540000,0.054467,0.481200,-1.2
11925,A.C. Green,24.0,82.0,64.0,32.1,0.503,0.0,0.000,7.8,0.505,0.503,4.6,0.773,3.0,8.7,1.1,1.1,0.5,1.5,2.5,11.4,LAL,1988/1989,PF,6-9,220.0,2.0,14.5,0.581,0.003,4.5,1.6,1.0,12.9,14.7,4.5,7.9,0.144,0.3,0.4,1.5,15.3,205.74,F,0.0,1.0,0.0,13.950000,-3.3,0.235714,-1.4,-0.021429,0.185714,0.536929,9.314286,-2.3,14.528571,0.058143,0.479643,3.3
11600,A.C. Green,25.0,82.0,82.0,30.6,0.529,0.2,0.235,9.0,0.536,0.532,4.4,0.786,3.1,9.0,1.3,1.1,0.7,1.5,2.1,13.3,LAL,1989/1990,PF,6-9,220.0,3.0,17.8,0.594,0.022,5.5,1.8,1.2,11.5,17.0,5.8,9.4,0.179,1.6,1.8,2.3,16.4,205.74,F,0.0,1.0,0.0,13.653846,3.1,0.507692,1.5,-0.161538,0.307692,0.532538,9.007692,-0.1,14.338462,0.060923,0.475846,-3.1
11264,A.C. Green,26.0,82.0,82.0,33.0,0.478,0.6,0.283,9.3,0.489,0.486,4.5,0.751,3.2,8.7,1.1,0.8,0.6,1.4,2.5,12.9,LAL,1990/1991,PF,6-9,220.0,4.0,14.7,0.548,0.057,4.6,1.2,1.1,10.7,17.1,4.4,7.7,0.137,-0.6,0.3,1.0,15.1,205.74,F,0.0,1.0,0.0,13.566667,0.9,0.291667,0.2,-0.200000,0.208333,0.531250,8.683333,1.5,14.108333,0.061250,0.475667,-0.9
10926,A.C. Green,27.0,82.0,21.0,26.4,0.476,0.7,0.200,5.9,0.507,0.486,3.7,0.738,2.5,6.3,0.9,0.7,0.3,1.2,1.4,9.1,LAL,1991/1992,PF,6-9,220.0,5.0,13.8,0.556,0.101,4.5,1.4,0.6,12.8,15.6,3.2,6.1,0.135,0.0,0.1,1.1,13.8,205.74,F,0.0,1.0,0.0,13.545455,-2.9,0.236364,-1.6,-0.227273,0.209091,0.529000,8.645455,-1.2,13.972727,0.057636,0.475636,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,Žarko Čabarkapa,23.0,37.0,0.0,12.5,0.482,1.0,0.361,3.6,0.515,0.521,1.7,0.813,0.9,2.7,0.7,0.3,0.1,0.8,1.6,6.2,GSW,2005/2006,PF,6-11,235.0,1.0,16.5,0.578,0.212,9.2,1.1,0.8,13.1,21.7,0.8,1.2,0.124,-0.5,0.1,0.2,11.6,210.82,F,0.0,1.0,0.0,16.500000,0.0,0.000000,0.0,0.100000,0.000000,0.578000,6.200000,0.0,21.700000,0.212000,0.482000,-6.6
8282,Željko Rebrača,29.0,74.0,4.0,15.9,0.505,0.0,0.000,5.1,0.505,0.505,2.4,0.771,1.1,3.9,0.5,0.4,1.0,1.1,2.6,6.9,DET,2002/2003,C,7-0,257.0,0.0,16.2,0.569,0.000,5.7,1.3,4.4,15.7,21.4,1.4,3.3,0.134,-2.3,-2.9,-0.1,14.5,213.36,C,1.0,0.0,0.0,12.350000,5.5,1.100000,0.8,-2.700000,-0.600000,0.608500,5.250000,5.9,15.750000,0.000000,0.555000,-2.2
8293,Željko Rebrača,30.0,30.0,12.0,16.3,0.552,0.0,0.000,4.8,0.552,0.552,1.6,0.792,0.9,3.1,0.3,0.2,0.6,1.0,2.6,6.6,DET,2003/2004,C,7-0,257.0,1.0,14.0,0.596,0.000,3.6,0.7,2.6,14.9,19.2,0.7,1.3,0.133,-2.6,-2.8,-0.1,11.2,213.36,C,1.0,0.0,0.0,13.633333,-2.2,0.000000,0.1,-2.766667,-0.366667,0.595333,5.800000,-2.2,17.633333,0.000000,0.538333,-4.4
8308,Željko Rebrača,31.0,21.0,2.0,10.6,0.407,0.0,0.000,2.6,0.407,0.407,1.3,0.786,0.9,2.3,0.2,0.2,0.4,0.6,2.1,3.1,DET,2004/2005,C,7-0,257.0,2.0,9.6,0.498,0.000,3.0,1.2,3.1,16.4,16.7,0.1,0.5,0.115,-3.9,-5.1,-0.1,12.8,213.36,C,1.0,0.0,0.0,13.725000,-4.4,-1.100000,-2.3,-2.775000,-0.850000,0.595500,6.000000,-2.5,18.025000,0.000000,0.541750,4.4


After all our feature engineering and dropping the rows with N/A target values, we now have 7881 rows and 54 columns. We have now finished with our data collection, data cleaning and data preprocessing. We also improved our data with more feature enginneering. Therefore we will move on to some EDAs, which will be the next notebook.

In [132]:
# I am gonna save my dataframe in a new csv file
numerical_df.to_csv("numerical_df.csv", index = False)